# Query Execution Notebook

In [ ]:
%pip install requests lxml nbformat

## anatomy_query
Description: fetch Individual instances from Class ID list

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (primary:Class) WHERE primary.short_form in $ids WITH primary CALL apoc.cypher.run('WITH primary OPTIONAL MATCH (primary)<- [:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual)<-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 5', {primary:primary}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, primary OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,''), iri: i.iri, types: labels(i), symbol: coalesce(i.symbol[0], '')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.symbol[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.symbol[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.symbol[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.symbol[0], '')} ,image_folder: COALESCE(irw.folder[0], ''), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }} }) END AS anatomy_channel_image ,primary RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])), symbol: coalesce(primary.symbol[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, 'bac066c' AS version, 'anatomy_query' AS query, anatomy_channel_image", "parameters" : { "ids" : ids }}
query_template = {'name': 'anatomy_query', 'description': 'fetch Individual instances from Class ID list', 'query': '"statement": "MATCH (primary:Class) WHERE primary.short_form in $ids WITH primary CALL apoc.cypher.run(\'WITH primary OPTIONAL MATCH (primary)<- [:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual)<-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 5\', {primary:primary}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, primary OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,\'\'), iri: i.iri, types: labels(i), symbol: coalesce(i.symbol[0], \'\')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.symbol[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.symbol[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.symbol[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.symbol[0], \'\')} ,image_folder: COALESCE(irw.folder[0], \'\'), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }} }) END AS anatomy_channel_image ,primary RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])), symbol: coalesce(primary.symbol[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, \'bac066c\' AS version, \'anatomy_query\' AS query, anatomy_channel_image", "parameters" : { "ids" : $ARRAY_ID_RESULTS }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## anat_image_query
Description: Get images for individual list

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (primary:Individual) WHERE primary.short_form in $ids WITH primary OPTIONAL MATCH (primary)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(template_anat:Individual) WITH template, channel, template_anat, irw, primary OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE collect ({ channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.symbol[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.symbol[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.symbol[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.symbol[0], '')} ,image_folder: COALESCE(irw.folder[0], ''), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }}) END AS channel_image,primary OPTIONAL MATCH (primary)-[:INSTANCEOF]->(typ:Class) WITH CASE WHEN typ is null THEN [] ELSE collect ({ short_form: typ.short_form, label: coalesce(typ.label,''), iri: typ.iri, types: labels(typ), symbol: coalesce(typ.symbol[0], '')} ) END AS types,primary,channel_image RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])), symbol: coalesce(primary.symbol[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, 'bac066c' AS version, 'anat_image_query' AS query, channel_image, types ", "parameters" : { "ids" : ids}}
query_template = {'name': 'anat_image_query', 'description': 'Get images for individual list', 'query': '"statement": "MATCH (primary:Individual) WHERE primary.short_form in $ids WITH primary OPTIONAL MATCH (primary)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(template_anat:Individual) WITH template, channel, template_anat, irw, primary OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE collect ({ channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.symbol[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.symbol[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.symbol[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.symbol[0], \'\')} ,image_folder: COALESCE(irw.folder[0], \'\'), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }}) END AS channel_image,primary OPTIONAL MATCH (primary)-[:INSTANCEOF]->(typ:Class) WITH CASE WHEN typ is null THEN [] ELSE collect ({ short_form: typ.short_form, label: coalesce(typ.label,\'\'), iri: typ.iri, types: labels(typ), symbol: coalesce(typ.symbol[0], \'\')} ) END AS types,primary,channel_image RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])), symbol: coalesce(primary.symbol[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, \'bac066c\' AS version, \'anat_image_query\' AS query, channel_image, types ", "parameters" : { "ids" : $ARRAY_ID_RESULTS}', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Get other cluster members
Description: $NAME's fellow cluster members

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (n:Neuron { short_form: $id } )-[r1:member_of]->(c:Cluster)-[r2:has_member]->(i:Neuron)<-[:depicts]-(j:Individual)-[k:in_register_with]->(m:Individual) OPTIONAL MATCH (i)-[:INSTANCEOF]->(ec:Class) RETURN DISTINCT i.short_form as id, CASE WHEN not i.synonym is null THEN i.label+replace(' ('+reduce(a='',n in i.synonym|a+n+', ')+')',', )',')') ELSE i.label END as name, i.description[0] as def, COLLECT(DISTINCT ec.label) as type, COLLECT(DISTINCT replace(k.folder[0],'http:','https:') + '/thumbnailT.png') as file", "parameters" : { "id" : "id" }}
query_template = {'name': 'Get other cluster members', 'description': "$NAME's fellow cluster members", 'query': '"statement": "MATCH (n:Neuron { short_form: $id } )-[r1:member_of]->(c:Cluster)-[r2:has_member]->(i:Neuron)<-[:depicts]-(j:Individual)-[k:in_register_with]->(m:Individual) OPTIONAL MATCH (i)-[:INSTANCEOF]->(ec:Class) RETURN DISTINCT i.short_form as id, CASE WHEN not i.synonym is null THEN i.label+replace(\' (\'+reduce(a=\'\',n in i.synonym|a+n+\', \')+\')\',\', )\',\')\') ELSE i.label END as name, i.description[0] as def, COLLECT(DISTINCT ec.label) as type, COLLECT(DISTINCT replace(k.folder[0],\'http:\',\'https:\') + \'/thumbnailT.png\') as file", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Fetch all example individuals for Class
Description: Fetch all example Individual instances of this Class or subclasses

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (c:Class)<-[:INSTANCEOF|SUBCLASSOF*..]-(primary:Individual)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(template_anat:Individual) WHERE c.short_form in [$id] WITH template, channel, template_anat, irw, primary OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE collect ({ channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.symbol[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.symbol[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.symbol[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.symbol[0], '')} ,image_folder: COALESCE(irw.folder[0], ''), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }}) END AS channel_image,primary OPTIONAL MATCH (primary)-[:INSTANCEOF]->(typ:Class) WITH CASE WHEN typ is null THEN [] ELSE collect ({ short_form: typ.short_form, label: coalesce(typ.label,''), iri: typ.iri, types: labels(typ), symbol: coalesce(typ.symbol[0], '')} ) END AS types,primary,channel_image RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])), symbol: coalesce(primary.symbol[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, 'm20210224' AS version, 'EXAMPLES_anat_image_query' AS query, channel_image, types", "parameters" : { "id" : "id" }}
query_template = {'name': 'Fetch all example individuals for Class', 'description': 'Fetch all example Individual instances of this Class or subclasses', 'query': '"statement": "MATCH (c:Class)<-[:INSTANCEOF|SUBCLASSOF*..]-(primary:Individual)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(template_anat:Individual) WHERE c.short_form in [$id] WITH template, channel, template_anat, irw, primary OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE collect ({ channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.symbol[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.symbol[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.symbol[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.symbol[0], \'\')} ,image_folder: COALESCE(irw.folder[0], \'\'), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }}) END AS channel_image,primary OPTIONAL MATCH (primary)-[:INSTANCEOF]->(typ:Class) WITH CASE WHEN typ is null THEN [] ELSE collect ({ short_form: typ.short_form, label: coalesce(typ.label,\'\'), iri: typ.iri, types: labels(typ), symbol: coalesce(typ.symbol[0], \'\')} ) END AS types,primary,channel_image RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])), symbol: coalesce(primary.symbol[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, \'m20210224\' AS version, \'EXAMPLES_anat_image_query\' AS query, channel_image, types", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Find domain individuals for template id
Description: Find domain individuals for template id

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (n:Template {short_form:$id})<-[:depicts]-(:Template)<-[r:in_register_with]-(dc:Individual)-[:depicts]->(di:Individual)-[:INSTANCEOF]->(d:Class) WHERE exists(r.index) RETURN distinct di.short_form as id, di.label as name, coalesce(di.description[0],d.description[0]) as def, COLLECT(DISTINCT d.label) as type, replace(r.folder[0],'http:','https:') + '/thumbnailT.png' as file", "parameters" : { "id" : "id" }}
query_template = {'name': 'Find domain individuals for template id', 'description': 'Find domain individuals for template id', 'query': '"statement": "MATCH (n:Template {short_form:$id})<-[:depicts]-(:Template)<-[r:in_register_with]-(dc:Individual)-[:depicts]->(di:Individual)-[:INSTANCEOF]->(d:Class) WHERE exists(r.index) RETURN distinct di.short_form as id, di.label as name, coalesce(di.description[0],d.description[0]) as def, COLLECT(DISTINCT d.label) as type, replace(r.folder[0],\'http:\',\'https:\') + \'/thumbnailT.png\' as file", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Get cluster members
Description: $NAME's members

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (c:Cluster { short_form: $id } )-[r2:has_member]->(i:Neuron)<-[:depicts]-(j:Individual)-[k:in_register_with]->(m:Individual) OPTIONAL MATCH (i)-[:INSTANCEOF]->(ec:Class) RETURN DISTINCT i.short_form as id, CASE WHEN not i.synonym is null THEN i.label+replace(' ('+reduce(a='',n in i.synonym|a+n+', ')+')',', )',')') ELSE i.label END as name, i.description[0] as def, COLLECT(DISTINCT ec.label) as type, COLLECT(DISTINCT replace(k.folder[0],'http:','https:') + '/thumbnailT.png') as file", "parameters" : { "id" : "id" }}
query_template = {'name': 'Get cluster members', 'description': "$NAME's members", 'query': '"statement": "MATCH (c:Cluster { short_form: $id } )-[r2:has_member]->(i:Neuron)<-[:depicts]-(j:Individual)-[k:in_register_with]->(m:Individual) OPTIONAL MATCH (i)-[:INSTANCEOF]->(ec:Class) RETURN DISTINCT i.short_form as id, CASE WHEN not i.synonym is null THEN i.label+replace(\' (\'+reduce(a=\'\',n in i.synonym|a+n+\', \')+\')\',\', )\',\')\') ELSE i.label END as name, i.description[0] as def, COLLECT(DISTINCT ec.label) as type, COLLECT(DISTINCT replace(k.folder[0],\'http:\',\'https:\') + \'/thumbnailT.png\') as file", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Find images for dataset
Description: Find images for a dataset

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (c:DataSet)<-[:has_source]-(primary:Individual)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(template_anat:Individual) WHERE c.short_form in [$id] WITH template, channel, template_anat, irw, primary OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE collect ({ channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.symbol[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.symbol[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.symbol[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.symbol[0], '')} ,image_folder: COALESCE(irw.folder[0], ''), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }}) END AS channel_image,primary OPTIONAL MATCH (primary)-[:INSTANCEOF]->(typ:Class) WITH CASE WHEN typ is null THEN [] ELSE collect ({ short_form: typ.short_form, label: coalesce(typ.label,''), iri: typ.iri, types: labels(typ), symbol: coalesce(typ.symbol[0], '')} ) END AS types,primary,channel_image RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])), symbol: coalesce(primary.symbol[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, 'm20210224' AS version, 'EXAMPLES_anat_image_query' AS query, channel_image, types", "parameters" : { "id" : "id" }}
query_template = {'name': 'Find images for dataset', 'description': 'Find images for a dataset', 'query': '"statement": "MATCH (c:DataSet)<-[:has_source]-(primary:Individual)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(template_anat:Individual) WHERE c.short_form in [$id] WITH template, channel, template_anat, irw, primary OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE collect ({ channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.symbol[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.symbol[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.symbol[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.symbol[0], \'\')} ,image_folder: COALESCE(irw.folder[0], \'\'), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }}) END AS channel_image,primary OPTIONAL MATCH (primary)-[:INSTANCEOF]->(typ:Class) WITH CASE WHEN typ is null THEN [] ELSE collect ({ short_form: typ.short_form, label: coalesce(typ.label,\'\'), iri: typ.iri, types: labels(typ), symbol: coalesce(typ.symbol[0], \'\')} ) END AS types,primary,channel_image RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])), symbol: coalesce(primary.symbol[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, \'m20210224\' AS version, \'EXAMPLES_anat_image_query\' AS query, channel_image, types", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Test Query for Exp from Anatomy
Description: Get JSON for anat_2_ep query

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (ep:Class:Expression_pattern)<-[ar:overlaps|part_of]-(:Individual)-[:INSTANCEOF]->(anat:Class) WHERE anat.short_form in $ids WITH DISTINCT collect(DISTINCT ar.pub[0]) as pubs, anat, ep UNWIND pubs as p OPTIONAL MATCH (pub:pub { short_form: p}) WITH anat, ep, collect({ core: { short_form: pub.short_form, label: coalesce(pub.label,''), iri: pub.iri, types: labels(pub), symbol: coalesce(pub.`symbol`[0], '')} , PubMed: coalesce(pub.PMID[0], ''), FlyBase: coalesce(([]+pub.FlyBase)[0], ''), DOI: coalesce(pub.DOI[0], '') }) as pubs CALL apoc.cypher.run('WITH ep OPTIONAL MATCH (ep)<- [:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual)<-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 5', {ep:ep}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, anat, ep, pubs OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class)  WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,''), iri: i.iri, types: labels(i), symbol: coalesce(i.`symbol`[0], '')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.`symbol`[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.`symbol`[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.`symbol`[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.`symbol`[0], '')} ,image_folder: COALESCE(irw.folder[0], ''), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }} }) END AS anatomy_channel_image ,anat,ep,pubs RETURN { short_form: anat.short_form, label: coalesce(anat.label,''), iri: anat.iri, types: labels(anat), symbol: coalesce(anat.`symbol`[0], '')}  as anatomy, { short_form: ep.short_form, label: coalesce(ep.label,''), iri: ep.iri, types: labels(ep), symbol: coalesce(ep.`symbol`[0], '')}  AS expression_pattern, 'Get JSON for anat_2_ep query' AS query, 'fe226a6' AS version , pubs, anatomy_channel_image", "parameters" : { "ids" : ids}}
query_template = {'name': 'Test Query for Exp from Anatomy', 'description': 'Get JSON for anat_2_ep query', 'query': '"statement": "MATCH (ep:Class:Expression_pattern)<-[ar:overlaps|part_of]-(:Individual)-[:INSTANCEOF]->(anat:Class) WHERE anat.short_form in $ids WITH DISTINCT collect(DISTINCT ar.pub[0]) as pubs, anat, ep UNWIND pubs as p OPTIONAL MATCH (pub:pub { short_form: p}) WITH anat, ep, collect({ core: { short_form: pub.short_form, label: coalesce(pub.label,\'\'), iri: pub.iri, types: labels(pub), symbol: coalesce(pub.`symbol`[0], \'\')} , PubMed: coalesce(pub.PMID[0], \'\'), FlyBase: coalesce(([]+pub.FlyBase)[0], \'\'), DOI: coalesce(pub.DOI[0], \'\') }) as pubs CALL apoc.cypher.run(\'WITH ep OPTIONAL MATCH (ep)<- [:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual)<-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 5\', {ep:ep}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, anat, ep, pubs OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class)  WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,\'\'), iri: i.iri, types: labels(i), symbol: coalesce(i.`symbol`[0], \'\')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.`symbol`[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.`symbol`[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.`symbol`[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.`symbol`[0], \'\')} ,image_folder: COALESCE(irw.folder[0], \'\'), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }} }) END AS anatomy_channel_image ,anat,ep,pubs RETURN { short_form: anat.short_form, label: coalesce(anat.label,\'\'), iri: anat.iri, types: labels(anat), symbol: coalesce(anat.`symbol`[0], \'\')}  as anatomy, { short_form: ep.short_form, label: coalesce(ep.label,\'\'), iri: ep.iri, types: labels(ep), symbol: coalesce(ep.`symbol`[0], \'\')}  AS expression_pattern, \'Get JSON for anat_2_ep query\' AS query, \'fe226a6\' AS version , pubs, anatomy_channel_image", "parameters" : { "ids" : $ARRAY_ID_RESULTS}', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Query for Anatomy from Exp
Description: Get JSON for ep_2_anat query

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (ep:Expression_pattern:Class)<-[ar:overlaps|part_of]-(anoni:Individual)-[:INSTANCEOF]->(anat:Class) WHERE ep.short_form in $ids WITH anoni, anat, ar OPTIONAL MATCH (p:pub { short_form: []+ar.pub[0]}) WITH anat, anoni, { core: { short_form: p.short_form, label: coalesce(p.label,''), iri: p.iri, types: labels(p), symbol: coalesce(p.symbol[0], '')} , PubMed: coalesce(p.PMID[0], ''), FlyBase: coalesce(p.FlyBase[0], ''), DOI: coalesce(p.DOI[0], '') } AS pub OPTIONAL MATCH (anoni)-[r:Related]->(o:FBdv) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r) } , object: { short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets,[])), symbol: coalesce(o.symbol[0], '')} }) END AS stages ,anoni,anat,pub CALL apoc.cypher.run('WITH anat OPTIONAL MATCH (anat)<- [:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual)<-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 10', {anat:anat}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, anoni, anat, pub, stages OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,''), iri: i.iri, types: labels(i), symbol: coalesce(i.symbol[0], '')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.symbol[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.symbol[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.symbol[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.symbol[0], '')} ,image_folder: COALESCE(irw.folder[0], ''), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }} }) END AS anatomy_channel_image ,anoni,anat,pub,stages RETURN { short_form: anat.short_form, label: coalesce(anat.label,''), iri: anat.iri, types: labels(anat), symbol: coalesce(anat.symbol[0], '')} AS anatomy, 'Get JSON for ep_2_anat query' AS query, 'bac066c' AS version , pub, stages, anatomy_channel_image", "parameters" : { "ids" : ids}}
query_template = {'name': 'Query for Anatomy from Exp', 'description': 'Get JSON for ep_2_anat query', 'query': '"statement": "MATCH (ep:Expression_pattern:Class)<-[ar:overlaps|part_of]-(anoni:Individual)-[:INSTANCEOF]->(anat:Class) WHERE ep.short_form in $ids WITH anoni, anat, ar OPTIONAL MATCH (p:pub { short_form: []+ar.pub[0]}) WITH anat, anoni, { core: { short_form: p.short_form, label: coalesce(p.label,\'\'), iri: p.iri, types: labels(p), symbol: coalesce(p.symbol[0], \'\')} , PubMed: coalesce(p.PMID[0], \'\'), FlyBase: coalesce(p.FlyBase[0], \'\'), DOI: coalesce(p.DOI[0], \'\') } AS pub OPTIONAL MATCH (anoni)-[r:Related]->(o:FBdv) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r) } , object: { short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets,[])), symbol: coalesce(o.symbol[0], \'\')} }) END AS stages ,anoni,anat,pub CALL apoc.cypher.run(\'WITH anat OPTIONAL MATCH (anat)<- [:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual)<-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 10\', {anat:anat}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, anoni, anat, pub, stages OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,\'\'), iri: i.iri, types: labels(i), symbol: coalesce(i.symbol[0], \'\')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.symbol[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.symbol[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.symbol[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.symbol[0], \'\')} ,image_folder: COALESCE(irw.folder[0], \'\'), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }} }) END AS anatomy_channel_image ,anoni,anat,pub,stages RETURN { short_form: anat.short_form, label: coalesce(anat.label,\'\'), iri: anat.iri, types: labels(anat), symbol: coalesce(anat.symbol[0], \'\')} AS anatomy, \'Get JSON for ep_2_anat query\' AS query, \'bac066c\' AS version , pub, stages, anatomy_channel_image", "parameters" : { "ids" : $ARRAY_ID_RESULTS}', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Test Query for Anatomy from Exp
Description: Get JSON for ep_2_anat query

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (ep:Expression_pattern:Class)<-[ar:overlaps|part_of]-(anoni:Individual)-[:INSTANCEOF]->(anat:Class) WHERE ep.short_form in [$id] WITH  anoni, anat, ar OPTIONAL MATCH (p:pub { short_form: []+ar.pub[0]}) WITH anat, anoni, { core: { short_form: p.short_form, label: coalesce(p.label,''), iri: p.iri, types: labels(p), symbol: coalesce(p.`symbol`[0], '')} , PubMed: coalesce(p.PMID[0], ''), FlyBase: coalesce(p.FlyBase[0], ''), DOI: coalesce(p.DOI[0], '') } AS pub OPTIONAL MATCH (anoni)-[r:Related]->(o:FBdv) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r) } , object: { short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets,[])), symbol: coalesce(o.`symbol`[0], '')}  }) END AS stages ,anoni,anat,pub CALL apoc.cypher.run('WITH anat OPTIONAL MATCH (anat)<- [:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual)<-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 10', {anat:anat}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, anoni, anat, pub, stages OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,''), iri: i.iri, types: labels(i), symbol: coalesce(i.`symbol`[0], '')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.`symbol`[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.`symbol`[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.`symbol`[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.`symbol`[0], '')} ,image_folder: COALESCE(irw.folder[0], ''), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }} }) END AS anatomy_channel_image ,anoni,anat,pub,stages RETURN { short_form: anat.short_form, label: coalesce(anat.label,''), iri: anat.iri, types: labels(anat), symbol: coalesce(anat.`symbol`[0], '')}  AS anatomy, 'Get JSON for ep_2_anat query' AS query, '3f881fe' AS version , pub, stages, anatomy_channel_image", "parameters" : { "id" : "id"}}
query_template = {'name': 'Test Query for Anatomy from Exp', 'description': 'Get JSON for ep_2_anat query', 'query': '"statement": "MATCH (ep:Expression_pattern:Class)<-[ar:overlaps|part_of]-(anoni:Individual)-[:INSTANCEOF]->(anat:Class) WHERE ep.short_form in [$id] WITH  anoni, anat, ar OPTIONAL MATCH (p:pub { short_form: []+ar.pub[0]}) WITH anat, anoni, { core: { short_form: p.short_form, label: coalesce(p.label,\'\'), iri: p.iri, types: labels(p), symbol: coalesce(p.`symbol`[0], \'\')} , PubMed: coalesce(p.PMID[0], \'\'), FlyBase: coalesce(p.FlyBase[0], \'\'), DOI: coalesce(p.DOI[0], \'\') } AS pub OPTIONAL MATCH (anoni)-[r:Related]->(o:FBdv) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r) } , object: { short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets,[])), symbol: coalesce(o.`symbol`[0], \'\')}  }) END AS stages ,anoni,anat,pub CALL apoc.cypher.run(\'WITH anat OPTIONAL MATCH (anat)<- [:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual)<-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 10\', {anat:anat}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, anoni, anat, pub, stages OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,\'\'), iri: i.iri, types: labels(i), symbol: coalesce(i.`symbol`[0], \'\')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.`symbol`[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.`symbol`[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.`symbol`[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.`symbol`[0], \'\')} ,image_folder: COALESCE(irw.folder[0], \'\'), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }} }) END AS anatomy_channel_image ,anoni,anat,pub,stages RETURN { short_form: anat.short_form, label: coalesce(anat.label,\'\'), iri: anat.iri, types: labels(anat), symbol: coalesce(anat.`symbol`[0], \'\')}  AS anatomy, \'Get JSON for ep_2_anat query\' AS query, \'3f881fe\' AS version , pub, stages, anatomy_channel_image", "parameters" : { "id" : "$ID"}', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Query for Exp from Anatomy
Description: Get JSON for anat_2_ep query

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (ep:Class:Expression_pattern)<-[ar:overlaps|part_of]-(:Individual)-[:INSTANCEOF]->(anat:Class) WHERE anat.short_form in $ids WITH DISTINCT collect(DISTINCT ar.pub[0]) as pubs, anat, ep UNWIND pubs as p OPTIONAL MATCH (pub:pub { short_form: p}) WITH anat, ep, collect({ core: { short_form: pub.short_form, label: coalesce(pub.label,''), iri: pub.iri, types: labels(pub), symbol: coalesce(pub.`symbol`[0], '')} , PubMed: coalesce(pub.PMID[0], ''), FlyBase: coalesce(([]+pub.FlyBase)[0], ''), DOI: coalesce(pub.DOI[0], '') }) as pubs CALL apoc.cypher.run('WITH ep OPTIONAL MATCH (ep)<- [:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual)<-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 5', {ep:ep}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, anat, ep, pubs OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class)  WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,''), iri: i.iri, types: labels(i), symbol: coalesce(i.`symbol`[0], '')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.`symbol`[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.`symbol`[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.`symbol`[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.`symbol`[0], '')} ,image_folder: COALESCE(irw.folder[0], ''), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }} }) END AS anatomy_channel_image ,anat,ep,pubs RETURN { short_form: anat.short_form, label: coalesce(anat.label,''), iri: anat.iri, types: labels(anat), symbol: coalesce(anat.`symbol`[0], '')}  as anatomy, { short_form: ep.short_form, label: coalesce(ep.label,''), iri: ep.iri, types: labels(ep), symbol: coalesce(ep.`symbol`[0], '')}  AS expression_pattern, 'Get JSON for anat_2_ep query' AS query, 'fe226a6' AS version , pubs, anatomy_channel_image", "parameters" : { "ids" : ids}}
query_template = {'name': 'Query for Exp from Anatomy', 'description': 'Get JSON for anat_2_ep query', 'query': '"statement": "MATCH (ep:Class:Expression_pattern)<-[ar:overlaps|part_of]-(:Individual)-[:INSTANCEOF]->(anat:Class) WHERE anat.short_form in $ids WITH DISTINCT collect(DISTINCT ar.pub[0]) as pubs, anat, ep UNWIND pubs as p OPTIONAL MATCH (pub:pub { short_form: p}) WITH anat, ep, collect({ core: { short_form: pub.short_form, label: coalesce(pub.label,\'\'), iri: pub.iri, types: labels(pub), symbol: coalesce(pub.`symbol`[0], \'\')} , PubMed: coalesce(pub.PMID[0], \'\'), FlyBase: coalesce(([]+pub.FlyBase)[0], \'\'), DOI: coalesce(pub.DOI[0], \'\') }) as pubs CALL apoc.cypher.run(\'WITH ep OPTIONAL MATCH (ep)<- [:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual)<-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 5\', {ep:ep}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, anat, ep, pubs OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class)  WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,\'\'), iri: i.iri, types: labels(i), symbol: coalesce(i.`symbol`[0], \'\')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.`symbol`[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.`symbol`[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.`symbol`[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.`symbol`[0], \'\')} ,image_folder: COALESCE(irw.folder[0], \'\'), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }} }) END AS anatomy_channel_image ,anat,ep,pubs RETURN { short_form: anat.short_form, label: coalesce(anat.label,\'\'), iri: anat.iri, types: labels(anat), symbol: coalesce(anat.`symbol`[0], \'\')}  as anatomy, { short_form: ep.short_form, label: coalesce(ep.label,\'\'), iri: ep.iri, types: labels(ep), symbol: coalesce(ep.`symbol`[0], \'\')}  AS expression_pattern, \'Get JSON for anat_2_ep query\' AS query, \'fe226a6\' AS version , pubs, anatomy_channel_image", "parameters" : { "ids" : $ARRAY_ID_RESULTS}', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## test_all_datasets_query
Description: Get JSON for template_2_datasets query

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (t:Template)<-[depicts]-(tc:Template)-[:in_register_with]-(c:Individual)-[:depicts]->(ai:Individual)-[:has_source]->(ds:DataSet) WITH distinct ds CALL apoc.cypher.run('WITH ds OPTIONAL MATCH (ds) <-[:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual) <-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 5', {ds:ds}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, ds OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,''), iri: i.iri, types: labels(i), symbol: coalesce(i.symbol[0], '')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.symbol[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.symbol[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.symbol[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.symbol[0], '')} ,image_folder: COALESCE(irw.folder[0], ''), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }} }) END AS anatomy_channel_image ,ds OPTIONAL MATCH (ds)-[rp:has_reference]->(p:pub) WITH CASE WHEN p is null THEN [] ELSE collect({ core: { short_form: p.short_form, label: coalesce(p.label,''), iri: p.iri, types: labels(p), symbol: coalesce(p.symbol[0], '')} , PubMed: coalesce(p.PMID[0], ''), FlyBase: coalesce(p.FlyBase[0], ''), DOI: coalesce(p.DOI[0], '') } ) END AS pubs,ds,anatomy_channel_image OPTIONAL MATCH (ds)-[:has_license|license]->(l:License) WITH collect ({ icon : coalesce(l.license_logo[0], ''), link : coalesce(l.license_url[0], ''), core : { short_form: l.short_form, label: coalesce(l.label,''), iri: l.iri, types: labels(l), unique_facets: apoc.coll.sort(coalesce(l.uniqueFacets,[])), symbol: coalesce(l.symbol[0], '')} }) as license,ds,anatomy_channel_image,pubs OPTIONAL MATCH (ds) <-[:has_source]-(i:Individual) WITH i, ds, anatomy_channel_image, pubs, license OPTIONAL MATCH (i)-[:INSTANCEOF]-(c:Class) WITH DISTINCT { images: count(distinct i),types: count(distinct c) } as dataset_counts,ds,anatomy_channel_image,pubs,license RETURN { short_form: ds.short_form, label: coalesce(ds.label,''), iri: ds.iri, types: labels(ds), symbol: coalesce(ds.symbol[0], '')} as dataset, 'bac066c' AS version, 'all_datasets_query' AS query, anatomy_channel_image, pubs, license, dataset_counts", "parameters" : { "id" : "id" }}
query_template = {'name': 'test_all_datasets_query', 'description': 'Get JSON for template_2_datasets query', 'query': '"statement": "MATCH (t:Template)<-[depicts]-(tc:Template)-[:in_register_with]-(c:Individual)-[:depicts]->(ai:Individual)-[:has_source]->(ds:DataSet) WITH distinct ds CALL apoc.cypher.run(\'WITH ds OPTIONAL MATCH (ds) <-[:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual) <-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 5\', {ds:ds}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, ds OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,\'\'), iri: i.iri, types: labels(i), symbol: coalesce(i.symbol[0], \'\')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.symbol[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.symbol[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.symbol[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.symbol[0], \'\')} ,image_folder: COALESCE(irw.folder[0], \'\'), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }} }) END AS anatomy_channel_image ,ds OPTIONAL MATCH (ds)-[rp:has_reference]->(p:pub) WITH CASE WHEN p is null THEN [] ELSE collect({ core: { short_form: p.short_form, label: coalesce(p.label,\'\'), iri: p.iri, types: labels(p), symbol: coalesce(p.symbol[0], \'\')} , PubMed: coalesce(p.PMID[0], \'\'), FlyBase: coalesce(p.FlyBase[0], \'\'), DOI: coalesce(p.DOI[0], \'\') } ) END AS pubs,ds,anatomy_channel_image OPTIONAL MATCH (ds)-[:has_license|license]->(l:License) WITH collect ({ icon : coalesce(l.license_logo[0], \'\'), link : coalesce(l.license_url[0], \'\'), core : { short_form: l.short_form, label: coalesce(l.label,\'\'), iri: l.iri, types: labels(l), unique_facets: apoc.coll.sort(coalesce(l.uniqueFacets,[])), symbol: coalesce(l.symbol[0], \'\')} }) as license,ds,anatomy_channel_image,pubs OPTIONAL MATCH (ds) <-[:has_source]-(i:Individual) WITH i, ds, anatomy_channel_image, pubs, license OPTIONAL MATCH (i)-[:INSTANCEOF]-(c:Class) WITH DISTINCT { images: count(distinct i),types: count(distinct c) } as dataset_counts,ds,anatomy_channel_image,pubs,license RETURN { short_form: ds.short_form, label: coalesce(ds.label,\'\'), iri: ds.iri, types: labels(ds), symbol: coalesce(ds.symbol[0], \'\')} as dataset, \'bac066c\' AS version, \'all_datasets_query\' AS query, anatomy_channel_image, pubs, license, dataset_counts", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Query for Exp from Anatomy
Description: Get JSON for anat_2_ep query

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (ep:Class:Expression_pattern)<-[ar:overlaps|part_of]-(:Individual)-[:INSTANCEOF]->(anat:Class) WHERE anat.short_form in $ids WITH DISTINCT collect(DISTINCT ar.pub[0]) as pubs, anat, ep UNWIND pubs as p OPTIONAL MATCH (pub:pub { short_form: p}) WITH anat, ep, collect({ core: { short_form: pub.short_form, label: coalesce(pub.label,''), iri: pub.iri, types: labels(pub), symbol: coalesce(pub.`symbol`[0], '')} , PubMed: coalesce(pub.PMID[0], ''), FlyBase: coalesce(([]+pub.FlyBase)[0], ''), DOI: coalesce(pub.DOI[0], '') }) as pubs CALL apoc.cypher.run('WITH ep OPTIONAL MATCH (ep)<- [:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual)<-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 5', {ep:ep}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, anat, ep, pubs OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class)  WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,''), iri: i.iri, types: labels(i), symbol: coalesce(i.`symbol`[0], '')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.`symbol`[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.`symbol`[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.`symbol`[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.`symbol`[0], '')} ,image_folder: COALESCE(irw.folder[0], ''), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }} }) END AS anatomy_channel_image ,anat,ep,pubs RETURN { short_form: anat.short_form, label: coalesce(anat.label,''), iri: anat.iri, types: labels(anat), symbol: coalesce(anat.`symbol`[0], '')}  as anatomy, { short_form: ep.short_form, label: coalesce(ep.label,''), iri: ep.iri, types: labels(ep), symbol: coalesce(ep.`symbol`[0], '')}  AS expression_pattern, 'Get JSON for anat_2_ep query' AS query, 'fe226a6' AS version , pubs, anatomy_channel_image", "parameters" : { "ids" : ids}}
query_template = {'name': 'Query for Exp from Anatomy', 'description': 'Get JSON for anat_2_ep query', 'query': '"statement": "MATCH (ep:Class:Expression_pattern)<-[ar:overlaps|part_of]-(:Individual)-[:INSTANCEOF]->(anat:Class) WHERE anat.short_form in $ids WITH DISTINCT collect(DISTINCT ar.pub[0]) as pubs, anat, ep UNWIND pubs as p OPTIONAL MATCH (pub:pub { short_form: p}) WITH anat, ep, collect({ core: { short_form: pub.short_form, label: coalesce(pub.label,\'\'), iri: pub.iri, types: labels(pub), symbol: coalesce(pub.`symbol`[0], \'\')} , PubMed: coalesce(pub.PMID[0], \'\'), FlyBase: coalesce(([]+pub.FlyBase)[0], \'\'), DOI: coalesce(pub.DOI[0], \'\') }) as pubs CALL apoc.cypher.run(\'WITH ep OPTIONAL MATCH (ep)<- [:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual)<-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 5\', {ep:ep}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, anat, ep, pubs OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class)  WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,\'\'), iri: i.iri, types: labels(i), symbol: coalesce(i.`symbol`[0], \'\')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.`symbol`[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.`symbol`[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.`symbol`[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.`symbol`[0], \'\')} ,image_folder: COALESCE(irw.folder[0], \'\'), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }} }) END AS anatomy_channel_image ,anat,ep,pubs RETURN { short_form: anat.short_form, label: coalesce(anat.label,\'\'), iri: anat.iri, types: labels(anat), symbol: coalesce(anat.`symbol`[0], \'\')}  as anatomy, { short_form: ep.short_form, label: coalesce(ep.label,\'\'), iri: ep.iri, types: labels(ep), symbol: coalesce(ep.`symbol`[0], \'\')}  AS expression_pattern, \'Get JSON for anat_2_ep query\' AS query, \'fe226a6\' AS version , pubs, anatomy_channel_image", "parameters" : { "ids" : $ARRAY_ID_RESULTS}', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## neuron_region_connectivity_query
Description: neuron_region_connectivity_query

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (primary) WHERE primary.short_form in [$id] WITH primary MATCH (target:Individual)<-[r:has_presynaptic_terminals_in|has_postsynaptic_terminal_in]-(primary) WITH DISTINCT collect(properties(r)) + {} as props, target, primary WITH apoc.map.removeKeys(apoc.map.merge(props[0], props[1]),['iri', 'short_form', 'Related', 'label', 'type']) as synapse_counts, { short_form: target.short_form, label: coalesce(target.label,''), iri: target.iri, types: labels(target), symbol: coalesce(target.`symbol`[0], '')} as object, target,primary OPTIONAL MATCH (o:Class)<-[r:SUBCLASSOF|INSTANCEOF]-(target) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets,[])), symbol: coalesce(o.`symbol`[0], '')} ) END AS parents ,primary,target,synapse_counts, object OPTIONAL MATCH (target)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(template_anat:Individual) WITH template, channel, template_anat, irw, primary, target, synapse_counts, object, parents   OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE collect ({ channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.`symbol`[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.`symbol`[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.`symbol`[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.`symbol`[0], '')} ,image_folder: COALESCE(irw.folder[0], ''), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }}) END AS channel_image,primary,target,synapse_counts, object,parents RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])), symbol: coalesce(primary.`symbol`[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, '64731dc' AS version , synapse_counts, object, parents, channel_image, 'neuron_region_connectivity_query' as query", "parameters" : { "id" : "id" }}
query_template = {'name': 'neuron_region_connectivity_query', 'description': 'neuron_region_connectivity_query', 'query': '"statement": "MATCH (primary) WHERE primary.short_form in [$id] WITH primary MATCH (target:Individual)<-[r:has_presynaptic_terminals_in|has_postsynaptic_terminal_in]-(primary) WITH DISTINCT collect(properties(r)) + {} as props, target, primary WITH apoc.map.removeKeys(apoc.map.merge(props[0], props[1]),[\'iri\', \'short_form\', \'Related\', \'label\', \'type\']) as synapse_counts, { short_form: target.short_form, label: coalesce(target.label,\'\'), iri: target.iri, types: labels(target), symbol: coalesce(target.`symbol`[0], \'\')} as object, target,primary OPTIONAL MATCH (o:Class)<-[r:SUBCLASSOF|INSTANCEOF]-(target) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets,[])), symbol: coalesce(o.`symbol`[0], \'\')} ) END AS parents ,primary,target,synapse_counts, object OPTIONAL MATCH (target)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(template_anat:Individual) WITH template, channel, template_anat, irw, primary, target, synapse_counts, object, parents   OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE collect ({ channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.`symbol`[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.`symbol`[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.`symbol`[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.`symbol`[0], \'\')} ,image_folder: COALESCE(irw.folder[0], \'\'), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }}) END AS channel_image,primary,target,synapse_counts, object,parents RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])), symbol: coalesce(primary.`symbol`[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, \'64731dc\' AS version , synapse_counts, object, parents, channel_image, \'neuron_region_connectivity_query\' as query", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## neuron_neuron_connectivity_query
Description: neuron_neuron_connectivity_query

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (primary) WHERE primary.short_form in [$id] WITH primary MATCH (oi:Individual)-[r:synapsed_to]-(primary:Individual) WHERE exists(r.weight) AND r.weight[0] > 1 WITH primary, oi OPTIONAL MATCH (oi)<-[down:synapsed_to]-(primary) WITH down, oi, primary OPTIONAL MATCH (primary)<-[up:synapsed_to]-(oi) WITH { downstream: [coalesce(down.weight[0],0)], upstream:[coalesce(up.weight[0],0)] } as synapse_counts, { short_form: oi.short_form, label: coalesce(oi.label,''), iri: oi.iri, types: labels(oi), symbol: coalesce(oi.`symbol`[0], '')}  as object, oi,primary OPTIONAL MATCH (o:Class)<-[r:SUBCLASSOF|INSTANCEOF]-(oi) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets,[])), symbol: coalesce(o.`symbol`[0], '')} ) END AS parents ,primary,oi,synapse_counts, object OPTIONAL MATCH (oi)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(template_anat:Individual) WITH template, channel, template_anat, irw, primary, oi, synapse_counts, object, parents   OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE collect ({ channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.`symbol`[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.`symbol`[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.`symbol`[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.`symbol`[0], '')} ,image_folder: COALESCE(irw.folder[0], ''), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }}) END AS channel_image,primary,oi,synapse_counts, object,parents RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])), symbol: coalesce(primary.`symbol`[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, '64731dc' AS version , synapse_counts, object, parents, channel_image, 'neuron_neuron_connectivity_query' as query", "parameters" : { "id" : "id" }}
query_template = {'name': 'neuron_neuron_connectivity_query', 'description': 'neuron_neuron_connectivity_query', 'query': '"statement": "MATCH (primary) WHERE primary.short_form in [$id] WITH primary MATCH (oi:Individual)-[r:synapsed_to]-(primary:Individual) WHERE exists(r.weight) AND r.weight[0] > 1 WITH primary, oi OPTIONAL MATCH (oi)<-[down:synapsed_to]-(primary) WITH down, oi, primary OPTIONAL MATCH (primary)<-[up:synapsed_to]-(oi) WITH { downstream: [coalesce(down.weight[0],0)], upstream:[coalesce(up.weight[0],0)] } as synapse_counts, { short_form: oi.short_form, label: coalesce(oi.label,\'\'), iri: oi.iri, types: labels(oi), symbol: coalesce(oi.`symbol`[0], \'\')}  as object, oi,primary OPTIONAL MATCH (o:Class)<-[r:SUBCLASSOF|INSTANCEOF]-(oi) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets,[])), symbol: coalesce(o.`symbol`[0], \'\')} ) END AS parents ,primary,oi,synapse_counts, object OPTIONAL MATCH (oi)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(template_anat:Individual) WITH template, channel, template_anat, irw, primary, oi, synapse_counts, object, parents   OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE collect ({ channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.`symbol`[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.`symbol`[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.`symbol`[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.`symbol`[0], \'\')} ,image_folder: COALESCE(irw.folder[0], \'\'), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }}) END AS channel_image,primary,oi,synapse_counts, object,parents RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])), symbol: coalesce(primary.`symbol`[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, \'64731dc\' AS version , synapse_counts, object, parents, channel_image, \'neuron_neuron_connectivity_query\' as query", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## NBLAST_anat_image_query
Description: find has_similar_morphology_to relationships

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (n:Individual)-[nblast:has_similar_morphology_to]-(primary:Individual) WHERE n.short_form in [$id] AND exists(nblast.NBLAST_score) WITH primary, nblast OPTIONAL MATCH (primary)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(template_anat:Individual) WITH template, channel, template_anat, irw, primary, nblast OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE collect ({ channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.symbol[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.symbol[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.symbol[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.symbol[0], '')} ,image_folder: COALESCE(irw.folder[0], ''), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }}) END AS channel_image,primary, nblast OPTIONAL MATCH (primary)-[:INSTANCEOF]->(typ:Class) WITH CASE WHEN typ is null THEN [] ELSE collect ({ short_form: typ.short_form, label: coalesce(typ.label,''), iri: typ.iri, types: labels(typ), symbol: coalesce(typ.symbol[0], '')} ) END AS types,primary,channel_image, nblast RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])), symbol: coalesce(primary.symbol[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, nblast.NBLAST_score[0] as score, 'm20210225' AS version, 'NBLAST_anat_image_query' AS query, channel_image, types", "parameters" : { "id" : "id" }}
query_template = {'name': 'NBLAST_anat_image_query', 'description': 'find has_similar_morphology_to relationships', 'query': '"statement": "MATCH (n:Individual)-[nblast:has_similar_morphology_to]-(primary:Individual) WHERE n.short_form in [$id] AND exists(nblast.NBLAST_score) WITH primary, nblast OPTIONAL MATCH (primary)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(template_anat:Individual) WITH template, channel, template_anat, irw, primary, nblast OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE collect ({ channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.symbol[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.symbol[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.symbol[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.symbol[0], \'\')} ,image_folder: COALESCE(irw.folder[0], \'\'), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }}) END AS channel_image,primary, nblast OPTIONAL MATCH (primary)-[:INSTANCEOF]->(typ:Class) WITH CASE WHEN typ is null THEN [] ELSE collect ({ short_form: typ.short_form, label: coalesce(typ.label,\'\'), iri: typ.iri, types: labels(typ), symbol: coalesce(typ.symbol[0], \'\')} ) END AS types,primary,channel_image, nblast RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])), symbol: coalesce(primary.symbol[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, nblast.NBLAST_score[0] as score, \'m20210225\' AS version, \'NBLAST_anat_image_query\' AS query, channel_image, types", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Fetch all entities for pub
Description: Fetch all entities that have reference to pub

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (p:pub)<-[:has_reference]-(primary) WHERE p.short_form in [$id] WITH distinct primary CALL apoc.cypher.run('WITH primary OPTIONAL MATCH (primary)<-[:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual) WHERE primary:Class WITH coalesce(i,primary) as i OPTIONAL MATCH (i)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]-> (template_anat:Individual)  RETURN template, channel, template_anat, i, irw limit 5', {primary:primary}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, primary OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,''), iri: i.iri, types: labels(i), symbol: coalesce(i.symbol[0], '')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.symbol[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.symbol[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.symbol[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.symbol[0], '')} ,image_folder: COALESCE(irw.folder[0], ''), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }} }) END AS anatomy_channel_image ,primary RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])), symbol: coalesce(primary.symbol[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, 'm_bac066c' AS version, 'pub_from_anatomy_query' AS query, anatomy_channel_image", "parameters" : { "id" : "id" }}
query_template = {'name': 'Fetch all entities for pub', 'description': 'Fetch all entities that have reference to pub', 'query': '"statement": "MATCH (p:pub)<-[:has_reference]-(primary) WHERE p.short_form in [$id] WITH distinct primary CALL apoc.cypher.run(\'WITH primary OPTIONAL MATCH (primary)<-[:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual) WHERE primary:Class WITH coalesce(i,primary) as i OPTIONAL MATCH (i)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]-> (template_anat:Individual)  RETURN template, channel, template_anat, i, irw limit 5\', {primary:primary}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, primary OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,\'\'), iri: i.iri, types: labels(i), symbol: coalesce(i.symbol[0], \'\')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.symbol[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.symbol[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.symbol[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.symbol[0], \'\')} ,image_folder: COALESCE(irw.folder[0], \'\'), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }} }) END AS anatomy_channel_image ,primary RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])), symbol: coalesce(primary.symbol[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, \'m_bac066c\' AS version, \'pub_from_anatomy_query\' AS query, anatomy_channel_image", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## NBLAST_anat_image_query_exp
Description: find has_similar_morphology_to relationships

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (n:Individual)-[nblast:has_similar_morphology_to_part_of]-(primary:Individual) WHERE n.short_form in [$id] AND EXISTS(nblast.NBLAST_score) WITH primary, nblast OPTIONAL MATCH (c:Class)<-[:INSTANCEOF]-(primary) OPTIONAL MATCH (primary)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(template_anat:Individual) WITH template, channel, template_anat, irw, primary, nblast OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE collect ({ channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.symbol[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.symbol[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.symbol[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.symbol[0], '')} ,image_folder: COALESCE(irw.folder[0], ''), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }}) END AS channel_image,primary, nblast OPTIONAL MATCH (primary)-[:INSTANCEOF]->(typ:Class) WITH CASE WHEN typ is null THEN [] ELSE collect ({ short_form: typ.short_form, label: coalesce(typ.label,''), iri: typ.iri, types: labels(typ), symbol: coalesce(typ.symbol[0], '')} ) END AS types,primary,channel_image, nblast RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])), symbol: coalesce(primary.symbol[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, nblast.NBLAST_score[0] as score, 'm20220726' AS version, 'NBLASTexp_anat_image_query' AS query, channel_image, types", "parameters" : { "id" : "id" }}
query_template = {'name': 'NBLAST_anat_image_query_exp', 'description': 'find has_similar_morphology_to relationships', 'query': '"statement": "MATCH (n:Individual)-[nblast:has_similar_morphology_to_part_of]-(primary:Individual) WHERE n.short_form in [$id] AND EXISTS(nblast.NBLAST_score) WITH primary, nblast OPTIONAL MATCH (c:Class)<-[:INSTANCEOF]-(primary) OPTIONAL MATCH (primary)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(template_anat:Individual) WITH template, channel, template_anat, irw, primary, nblast OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE collect ({ channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.symbol[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.symbol[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.symbol[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.symbol[0], \'\')} ,image_folder: COALESCE(irw.folder[0], \'\'), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }}) END AS channel_image,primary, nblast OPTIONAL MATCH (primary)-[:INSTANCEOF]->(typ:Class) WITH CASE WHEN typ is null THEN [] ELSE collect ({ short_form: typ.short_form, label: coalesce(typ.label,\'\'), iri: typ.iri, types: labels(typ), symbol: coalesce(typ.symbol[0], \'\')} ) END AS types,primary,channel_image, nblast RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])), symbol: coalesce(primary.symbol[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, nblast.NBLAST_score[0] as score, \'m20220726\' AS version, \'NBLASTexp_anat_image_query\' AS query, channel_image, types", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## NB_anat_image_query
Description: find has_similar_morphology_to relationships

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (n:Individual)-[nblast:has_similar_morphology_to_part_of]-(primary:Individual) WHERE n.short_form in [$id] AND exists(nblast.neuronbridge_score)  WITH primary, nblast OPTIONAL MATCH (primary)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(template_anat:Individual) WITH template, channel, template_anat, irw, primary, nblast OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE collect ({ channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.symbol[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.symbol[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.symbol[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.symbol[0], '')} ,image_folder: COALESCE(irw.folder[0], ''), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }}) END AS channel_image,primary, nblast OPTIONAL MATCH (primary)-[:INSTANCEOF]->(typ:Class) WITH CASE WHEN typ is null THEN [] ELSE collect ({ short_form: typ.short_form, label: coalesce(typ.label,''), iri: typ.iri, types: labels(typ), symbol: coalesce(typ.symbol[0], '')} ) END AS types,primary,channel_image, nblast RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])), symbol: coalesce(primary.symbol[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, nblast.neuronbridge_score[0] as score, 'm20220726' AS version, 'NeuronBridge_anat_image_query' AS query, channel_image, types", "parameters" : { "id" : "id" }}
query_template = {'name': 'NB_anat_image_query', 'description': 'find has_similar_morphology_to relationships', 'query': '"statement": "MATCH (n:Individual)-[nblast:has_similar_morphology_to_part_of]-(primary:Individual) WHERE n.short_form in [$id] AND exists(nblast.neuronbridge_score)  WITH primary, nblast OPTIONAL MATCH (primary)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(template_anat:Individual) WITH template, channel, template_anat, irw, primary, nblast OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE collect ({ channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets,[])), symbol: coalesce(channel.symbol[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets,[])), symbol: coalesce(technique.symbol[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets,[])), symbol: coalesce(template.symbol[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), symbol: coalesce(template_anat.symbol[0], \'\')} ,image_folder: COALESCE(irw.folder[0], \'\'), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }}) END AS channel_image,primary, nblast OPTIONAL MATCH (primary)-[:INSTANCEOF]->(typ:Class) WITH CASE WHEN typ is null THEN [] ELSE collect ({ short_form: typ.short_form, label: coalesce(typ.label,\'\'), iri: typ.iri, types: labels(typ), symbol: coalesce(typ.symbol[0], \'\')} ) END AS types,primary,channel_image, nblast RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])), symbol: coalesce(primary.symbol[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, nblast.neuronbridge_score[0] as score, \'m20220726\' AS version, \'NeuronBridge_anat_image_query\' AS query, channel_image, types", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Get JSON for cluster expression query
Description: Get JSON for cluster expression query

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (primary:Individual:Cluster) WHERE primary.short_form in [$id] WITH primary MATCH (primary)-[e:expresses]->(g:Gene:Class) WITH coalesce(e.expression_level_padded[0], e.expression_level[0]) as expression_level, e.expression_extent[0] as expression_extent, { short_form: g.short_form, label: coalesce(g.label,''), iri: g.iri, types: labels(g), unique_facets: apoc.coll.sort(coalesce(g.uniqueFacets, [])), symbol: coalesce(([]+g.symbol)[0], '')}  AS gene,primary MATCH (a:Anatomy)<-[:composed_primarily_of]-(primary) WITH { short_form: a.short_form, label: coalesce(a.label,''), iri: a.iri, types: labels(a), unique_facets: apoc.coll.sort(coalesce(a.uniqueFacets, [])), symbol: coalesce(([]+a.symbol)[0], '')}  AS anatomy,primary,expression_level,expression_extent,gene  RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, 'Get JSON for cluster expression query' AS query, 'a3c0d68' AS version , expression_level, expression_extent, gene, anatomy", "parameters" : { "id" : "id" }}
query_template = {'name': 'Get JSON for cluster expression query', 'description': 'Get JSON for cluster expression query', 'query': '"statement": "MATCH (primary:Individual:Cluster) WHERE primary.short_form in [$id] WITH primary MATCH (primary)-[e:expresses]->(g:Gene:Class) WITH coalesce(e.expression_level_padded[0], e.expression_level[0]) as expression_level, e.expression_extent[0] as expression_extent, { short_form: g.short_form, label: coalesce(g.label,\'\'), iri: g.iri, types: labels(g), unique_facets: apoc.coll.sort(coalesce(g.uniqueFacets, [])), symbol: coalesce(([]+g.symbol)[0], \'\')}  AS gene,primary MATCH (a:Anatomy)<-[:composed_primarily_of]-(primary) WITH { short_form: a.short_form, label: coalesce(a.label,\'\'), iri: a.iri, types: labels(a), unique_facets: apoc.coll.sort(coalesce(a.uniqueFacets, [])), symbol: coalesce(([]+a.symbol)[0], \'\')}  AS anatomy,primary,expression_level,expression_extent,gene  RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, \'Get JSON for cluster expression query\' AS query, \'a3c0d68\' AS version , expression_level, expression_extent, gene, anatomy", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Get JSON for anat scRNAseq query
Description: Get JSON for anat scRNAseq query

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (primary:Class:Anatomy) WHERE primary.short_form in $ids OR primary.short_form = $id WITH primary MATCH (primary)<-[:composed_primarily_of]-(c:Cluster)-[:has_source]->(ds:scRNAseq_DataSet)OPTIONAL MATCH (ds)-[:has_reference]->(p:pub) WITH { short_form: c.short_form, label: coalesce(c.label,''), iri: c.iri, types: labels(c), unique_facets: apoc.coll.sort(coalesce(c.uniqueFacets, [])), symbol: coalesce(([]+c.symbol)[0], '')}  AS cluster, { short_form: ds.short_form, label: coalesce(ds.label,''), iri: ds.iri, types: labels(ds), unique_facets: apoc.coll.sort(coalesce(ds.uniqueFacets, [])), symbol: coalesce(([]+ds.symbol)[0], '')}  AS dataset, COLLECT({ core: { short_form: p.short_form, label: coalesce(p.label,''), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], '')} , PubMed: coalesce(([]+p.PMID)[0], ''), FlyBase: coalesce(([]+p.FlyBase)[0], ''), DOI: coalesce(([]+p.DOI)[0], '') }) AS pubs,primary RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, 'Get JSON for anat scRNAseq query' AS query, 'a3c0d68' AS version , cluster, dataset, pubs", "parameters" : { "ids" : ids , "id" : "id" }}
query_template = {'name': 'Get JSON for anat scRNAseq query', 'description': 'Get JSON for anat scRNAseq query', 'query': '"statement": "MATCH (primary:Class:Anatomy) WHERE primary.short_form in $ids OR primary.short_form = $id WITH primary MATCH (primary)<-[:composed_primarily_of]-(c:Cluster)-[:has_source]->(ds:scRNAseq_DataSet)OPTIONAL MATCH (ds)-[:has_reference]->(p:pub) WITH { short_form: c.short_form, label: coalesce(c.label,\'\'), iri: c.iri, types: labels(c), unique_facets: apoc.coll.sort(coalesce(c.uniqueFacets, [])), symbol: coalesce(([]+c.symbol)[0], \'\')}  AS cluster, { short_form: ds.short_form, label: coalesce(ds.label,\'\'), iri: ds.iri, types: labels(ds), unique_facets: apoc.coll.sort(coalesce(ds.uniqueFacets, [])), symbol: coalesce(([]+ds.symbol)[0], \'\')}  AS dataset, COLLECT({ core: { short_form: p.short_form, label: coalesce(p.label,\'\'), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], \'\')} , PubMed: coalesce(([]+p.PMID)[0], \'\'), FlyBase: coalesce(([]+p.FlyBase)[0], \'\'), DOI: coalesce(([]+p.DOI)[0], \'\') }) AS pubs,primary RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, \'Get JSON for anat scRNAseq query\' AS query, \'a3c0d68\' AS version , cluster, dataset, pubs", "parameters" : { "ids" : $ARRAY_ID_RESULTS , "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Get JSON for dataset scRNAseq query
Description: Get JSON for dataset scRNAseq query

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (c:Individual:Cluster)-[:has_source]->(ds:scRNAseq_DataSet) WHERE ds.short_form = $id MATCH (a:Class:Anatomy)<-[:composed_primarily_of]-(c) WITH *, { short_form: a.short_form, label: coalesce(a.label,''), iri: a.iri, types: labels(a), unique_facets: apoc.coll.sort(coalesce(a.uniqueFacets, [])), symbol: coalesce(([]+a.symbol)[0], '')}  AS anatomy OPTIONAL MATCH (ds)-[:has_reference]->(p:pub) WITH COLLECT({ core: { short_form: p.short_form, label: coalesce(p.label,''), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], '')} , PubMed: coalesce(([]+p.PMID)[0], ''), FlyBase: coalesce(([]+p.FlyBase)[0], ''), DOI: coalesce(([]+p.DOI)[0], '') }) AS pubs, c, anatomy RETURN { core : { short_form: c.short_form, label: coalesce(c.label,''), iri: c.iri, types: labels(c), unique_facets: apoc.coll.sort(coalesce(c.uniqueFacets, [])), symbol: coalesce(([]+c.symbol)[0], '')} , description : coalesce(c.description, []), comment : coalesce(c.comment, []) } AS term, anatomy, 'Get JSON for dataset scRNAseq query' AS query, 'm20240712' AS version, pubs", "parameters" : { "id" : "id" }}
query_template = {'name': 'Get JSON for dataset scRNAseq query', 'description': 'Get JSON for dataset scRNAseq query', 'query': '"statement": "MATCH (c:Individual:Cluster)-[:has_source]->(ds:scRNAseq_DataSet) WHERE ds.short_form = $id MATCH (a:Class:Anatomy)<-[:composed_primarily_of]-(c) WITH *, { short_form: a.short_form, label: coalesce(a.label,\'\'), iri: a.iri, types: labels(a), unique_facets: apoc.coll.sort(coalesce(a.uniqueFacets, [])), symbol: coalesce(([]+a.symbol)[0], \'\')}  AS anatomy OPTIONAL MATCH (ds)-[:has_reference]->(p:pub) WITH COLLECT({ core: { short_form: p.short_form, label: coalesce(p.label,\'\'), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], \'\')} , PubMed: coalesce(([]+p.PMID)[0], \'\'), FlyBase: coalesce(([]+p.FlyBase)[0], \'\'), DOI: coalesce(([]+p.DOI)[0], \'\') }) AS pubs, c, anatomy RETURN { core : { short_form: c.short_form, label: coalesce(c.label,\'\'), iri: c.iri, types: labels(c), unique_facets: apoc.coll.sort(coalesce(c.uniqueFacets, [])), symbol: coalesce(([]+c.symbol)[0], \'\')} , description : coalesce(c.description, []), comment : coalesce(c.comment, []) } AS term, anatomy, \'Get JSON for dataset scRNAseq query\' AS query, \'m20240712\' AS version, pubs", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Get JSON for cluster expression query
Description: Get JSON for cluster expression query

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (primary:Individual:Cluster)-[e:expresses]->(g:Gene:Class) WHERE g.short_form in [$id] WITH e.expression_level[0] as expression_level, e.expression_extent[0] as expression_extent, { short_form: g.short_form, label: coalesce(g.label,''), iri: g.iri, types: labels(g), unique_facets: apoc.coll.sort(coalesce(g.uniqueFacets, [])), symbol: coalesce(([]+g.symbol)[0], '')}  AS gene,primary MATCH (a:Anatomy)<-[:composed_primarily_of]-(primary) WITH { short_form: a.short_form, label: coalesce(a.label,''), iri: a.iri, types: labels(a), unique_facets: apoc.coll.sort(coalesce(a.uniqueFacets, [])), symbol: coalesce(([]+a.symbol)[0], '')}  AS anatomy,primary,expression_level,expression_extent,gene  RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, 'Get JSON for expression in cluster query' AS query, 'ma3c0d68' AS version , expression_level, expression_extent, anatomy", "parameters" : { "id" : "id" }}
query_template = {'name': 'Get JSON for cluster expression query', 'description': 'Get JSON for cluster expression query', 'query': '"statement": "MATCH (primary:Individual:Cluster)-[e:expresses]->(g:Gene:Class) WHERE g.short_form in [$id] WITH e.expression_level[0] as expression_level, e.expression_extent[0] as expression_extent, { short_form: g.short_form, label: coalesce(g.label,\'\'), iri: g.iri, types: labels(g), unique_facets: apoc.coll.sort(coalesce(g.uniqueFacets, [])), symbol: coalesce(([]+g.symbol)[0], \'\')}  AS gene,primary MATCH (a:Anatomy)<-[:composed_primarily_of]-(primary) WITH { short_form: a.short_form, label: coalesce(a.label,\'\'), iri: a.iri, types: labels(a), unique_facets: apoc.coll.sort(coalesce(a.uniqueFacets, [])), symbol: coalesce(([]+a.symbol)[0], \'\')}  AS anatomy,primary,expression_level,expression_extent,gene  RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, \'Get JSON for expression in cluster query\' AS query, \'ma3c0d68\' AS version , expression_level, expression_extent, anatomy", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Get term core info
Description: Fetches term core details.

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (primary:Entity {short_form: $id}) RETURN { core: { short_form: primary.short_form, label: primary.label, iri: primary.iri, types: labels(primary) }} as term", "parameters" : { "id" : "id" }}
query_template = {'name': 'Get term core info', 'description': 'Fetches term core details.', 'query': '"statement": "MATCH (primary:Entity {short_form: $id}) RETURN { core: { short_form: primary.short_form, label: primary.label, iri: primary.iri, types: labels(primary) }} as term", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Get baseline term info
Description: Fetches basic term details.

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (primary {short_form: $id }) WHERE NOT primary:pub AND NOT primary:License AND NOT primary:Individual AND NOT primary:Class AND NOT primary:Template AND NOT primary:DataSet OPTIONAL MATCH (o:Class)<-[r:SUBCLASSOF|INSTANCEOF]-(primary)  WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ short_form: o.short_form, label: o.label, iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets,[])) } ) END AS parents , primary OPTIONAL MATCH (o:Class)<-[r { type: 'Related' }]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.uri, type: type(r) } , object: { short_form: o.short_form, label: o.label, iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets,[])) }  }) END AS relationships , parents, primary OPTIONAL MATCH (s:Site)<-[dbx:hasDbXref]-(primary)  WITH CASE WHEN s IS NULL THEN [] ELSE COLLECT({ link: s.link_base + coalesce(dbx.accession, ''), link_text: s.label, site: { short_form: s.short_form, label: s.label, iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets,[])) }, icon: coalesce(s.link_icon_url[0], '') }) END AS xrefs, parents, relationships, primary OPTIONAL MATCH (primary)-[rp:has_reference { typ: 'syn'}]->(p:pub:Individual)  WITH CASE WHEN p is null THEN [] ELSE collect({ pub: { core: { short_form: p.short_form, label: p.label, iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets,[])) } , PubMed: coalesce(p.PMID, ''), FlyBase: coalesce(p.FlyBase, ''), DOI: coalesce(p.DOI, '') } , synonym: { label: coalesce(rp.synonym[0], ''), scope: coalesce(rp.scope, ''), type: coalesce(rp.cat,'') }  }) END AS pub_syn, parents, relationships, xrefs, primary OPTIONAL MATCH (primary)-[rp:has_reference { typ: 'def'}]->(p:pub:Individual) WITH CASE WHEN p is null THEN [] ELSE collect({ core: { short_form: p.short_form, label: p.label, iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets,[])) } , PubMed: coalesce(p.PMID, ''), FlyBase: coalesce(p.FlyBase, ''), DOI: coalesce(p.DOI, '') } ) END AS def_pubs, parents, relationships, xrefs, pub_syn, primary RETURN { core: { short_form: primary.short_form, label: primary.label, iri: primary.iri, types: labels(primary),  unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])) } , description: coalesce(primary.description, []), comment: coalesce(primary.`annotation-comment`, [])} as term ,parents,relationships,xrefs,pub_syn,def_pubs, 'Base' as query, 'manual' AS version", "parameters" : { "id" : "id" }}
query_template = {'name': 'Get baseline term info', 'description': 'Fetches basic term details.', 'query': '"statement": "MATCH (primary {short_form: $id }) WHERE NOT primary:pub AND NOT primary:License AND NOT primary:Individual AND NOT primary:Class AND NOT primary:Template AND NOT primary:DataSet OPTIONAL MATCH (o:Class)<-[r:SUBCLASSOF|INSTANCEOF]-(primary)  WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ short_form: o.short_form, label: o.label, iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets,[])) } ) END AS parents , primary OPTIONAL MATCH (o:Class)<-[r { type: \'Related\' }]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.uri, type: type(r) } , object: { short_form: o.short_form, label: o.label, iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets,[])) }  }) END AS relationships , parents, primary OPTIONAL MATCH (s:Site)<-[dbx:hasDbXref]-(primary)  WITH CASE WHEN s IS NULL THEN [] ELSE COLLECT({ link: s.link_base + coalesce(dbx.accession, \'\'), link_text: s.label, site: { short_form: s.short_form, label: s.label, iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets,[])) }, icon: coalesce(s.link_icon_url[0], \'\') }) END AS xrefs, parents, relationships, primary OPTIONAL MATCH (primary)-[rp:has_reference { typ: \'syn\'}]->(p:pub:Individual)  WITH CASE WHEN p is null THEN [] ELSE collect({ pub: { core: { short_form: p.short_form, label: p.label, iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets,[])) } , PubMed: coalesce(p.PMID, \'\'), FlyBase: coalesce(p.FlyBase, \'\'), DOI: coalesce(p.DOI, \'\') } , synonym: { label: coalesce(rp.synonym[0], \'\'), scope: coalesce(rp.scope, \'\'), type: coalesce(rp.cat,\'\') }  }) END AS pub_syn, parents, relationships, xrefs, primary OPTIONAL MATCH (primary)-[rp:has_reference { typ: \'def\'}]->(p:pub:Individual) WITH CASE WHEN p is null THEN [] ELSE collect({ core: { short_form: p.short_form, label: p.label, iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets,[])) } , PubMed: coalesce(p.PMID, \'\'), FlyBase: coalesce(p.FlyBase, \'\'), DOI: coalesce(p.DOI, \'\') } ) END AS def_pubs, parents, relationships, xrefs, pub_syn, primary RETURN { core: { short_form: primary.short_form, label: primary.label, iri: primary.iri, types: labels(primary),  unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets,[])) } , description: coalesce(primary.description, []), comment: coalesce(primary.`annotation-comment`, [])} as term ,parents,relationships,xrefs,pub_syn,def_pubs, \'Base\' as query, \'manual\' AS version", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Get JSON for Class
Description: Get JSON for Class

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (primary:Class) WHERE primary.short_form in [$id] WITH primary OPTIONAL MATCH (o:Class)<-[r:SUBCLASSOF|INSTANCEOF]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], '')} ) END AS parents ,primary OPTIONAL MATCH (o:Class)<-[r {}]-(primary) WHERE (r.type='Related') AND ((not exists(r.hide_in_terminfo)) OR (not (r.hide_in_terminfo[0] = true))) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r), database_cross_reference: coalesce(r.database_cross_reference, []), confidence_value: coalesce(toString(r.confidence_value[0]), '') }, object: { short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], '')} }) END AS relationships ,primary,parents OPTIONAL MATCH (o)<-[r:term_replaced_by]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r), database_cross_reference: coalesce(r.database_cross_reference, []), confidence_value: coalesce(toString(r.confidence_value[0]), '') }, object: { short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], '')} }) END AS related_individuals ,primary,parents,relationships OPTIONAL MATCH (s:Site { short_form: apoc.convert.toList(primary.self_xref)[0] }) WITH CASE WHEN s IS NULL THEN [] ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], ''), accession: coalesce(primary.short_form, ''), link_text: primary.label + ' on ' + s.label, homepage: coalesce(s.homepage[0], ''), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,''), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], '')} , icon: coalesce(s.link_icon_url[0], ''), link_postfix: coalesce(s.link_postfix[0], '')}) END AS self_xref, primary, parents, relationships, related_individuals OPTIONAL MATCH (s:Site)<-[dbx:database_cross_reference]-(primary) WITH CASE WHEN s IS NULL THEN self_xref ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], ''), accession: coalesce(([]+dbx.accession)[0], ''), link_text: primary.label + ' on ' + s.label, homepage: coalesce(s.homepage[0], ''), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,''), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], '')} , icon: coalesce(s.link_icon_url[0], ''), link_postfix: coalesce(s.link_postfix[0], '')}) + self_xref END AS xrefs,primary,parents,relationships,related_individuals CALL apoc.cypher.run('WITH primary OPTIONAL MATCH (primary)<- [:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual)<-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 10', {primary:primary}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, primary, parents, relationships, related_individuals, xrefs OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,''), iri: i.iri, types: labels(i), unique_facets: apoc.coll.sort(coalesce(i.uniqueFacets, [])), symbol: coalesce(([]+i.symbol)[0], '')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets, [])), symbol: coalesce(([]+channel.symbol)[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets, [])), symbol: coalesce(([]+technique.symbol)[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets, [])), symbol: coalesce(([]+template.symbol)[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), unique_facets: apoc.coll.sort(coalesce(template_anat.uniqueFacets, [])), symbol: coalesce(([]+template_anat.symbol)[0], '')} ,image_folder: COALESCE(([]+irw.folder)[0], ''), image_nrrd: COALESCE(([]+irw.nrrd)[0], ''), image_thumbnail: COALESCE(([]+irw.thumbnail)[0], ''), image_swc: COALESCE(([]+irw.swc)[0], ''), image_obj: COALESCE(([]+irw.obj)[0], ''), image_wlz: COALESCE(([]+irw.wlz)[0], ''), index: coalesce(apoc.convert.toInteger(([]+irw.index)[0]), []) + [] }} }) END AS anatomy_channel_image ,primary,parents,relationships,related_individuals,xrefs OPTIONAL MATCH (primary)-[rp:has_reference]->(p:pub) where rp.typ = 'syn' WITH CASE WHEN p is null THEN [] ELSE collect({ pub: { core: { short_form: p.short_form, label: coalesce(p.label,''), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], '')} , PubMed: coalesce(([]+p.PMID)[0], ''), FlyBase: coalesce(([]+p.FlyBase)[0], ''), DOI: coalesce(([]+p.DOI)[0], '') } , synonym: { label: coalesce(([]+rp.value)[0], ''), scope: coalesce(rp.scope, ''), type: coalesce(([]+rp.has_synonym_type)[0],'') } }) END AS pub_syn,primary,parents,relationships,related_individuals,xrefs,anatomy_channel_image OPTIONAL MATCH (primary)-[rp:has_reference]->(p:pub) WHERE rp.typ = 'def' WITH CASE WHEN p is null THEN [] ELSE collect({ core: { short_form: p.short_form, label: coalesce(p.label,''), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], '')} , PubMed: coalesce(([]+p.PMID)[0], ''), FlyBase: coalesce(([]+p.FlyBase)[0], ''), DOI: coalesce(([]+p.DOI)[0], '') } ) END AS def_pubs,primary,parents,relationships,related_individuals,xrefs,anatomy_channel_image,pub_syn RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, 'Get JSON for Class' AS query, 'c58c844' AS version , parents, relationships, related_individuals, xrefs, anatomy_channel_image, pub_syn, def_pubs", "parameters" : { "id" : "id" }}
query_template = {'name': 'Get JSON for Class', 'description': 'Get JSON for Class', 'query': '"statement": "MATCH (primary:Class) WHERE primary.short_form in [$id] WITH primary OPTIONAL MATCH (o:Class)<-[r:SUBCLASSOF|INSTANCEOF]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], \'\')} ) END AS parents ,primary OPTIONAL MATCH (o:Class)<-[r {}]-(primary) WHERE (r.type=\'Related\') AND ((not exists(r.hide_in_terminfo)) OR (not (r.hide_in_terminfo[0] = true))) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r), database_cross_reference: coalesce(r.database_cross_reference, []), confidence_value: coalesce(toString(r.confidence_value[0]), \'\') }, object: { short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], \'\')} }) END AS relationships ,primary,parents OPTIONAL MATCH (o)<-[r:term_replaced_by]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r), database_cross_reference: coalesce(r.database_cross_reference, []), confidence_value: coalesce(toString(r.confidence_value[0]), \'\') }, object: { short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], \'\')} }) END AS related_individuals ,primary,parents,relationships OPTIONAL MATCH (s:Site { short_form: apoc.convert.toList(primary.self_xref)[0] }) WITH CASE WHEN s IS NULL THEN [] ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], \'\'), accession: coalesce(primary.short_form, \'\'), link_text: primary.label + \' on \' + s.label, homepage: coalesce(s.homepage[0], \'\'), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,\'\'), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], \'\')} , icon: coalesce(s.link_icon_url[0], \'\'), link_postfix: coalesce(s.link_postfix[0], \'\')}) END AS self_xref, primary, parents, relationships, related_individuals OPTIONAL MATCH (s:Site)<-[dbx:database_cross_reference]-(primary) WITH CASE WHEN s IS NULL THEN self_xref ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], \'\'), accession: coalesce(([]+dbx.accession)[0], \'\'), link_text: primary.label + \' on \' + s.label, homepage: coalesce(s.homepage[0], \'\'), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,\'\'), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], \'\')} , icon: coalesce(s.link_icon_url[0], \'\'), link_postfix: coalesce(s.link_postfix[0], \'\')}) + self_xref END AS xrefs,primary,parents,relationships,related_individuals CALL apoc.cypher.run(\'WITH primary OPTIONAL MATCH (primary)<- [:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual)<-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 10\', {primary:primary}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, primary, parents, relationships, related_individuals, xrefs OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,\'\'), iri: i.iri, types: labels(i), unique_facets: apoc.coll.sort(coalesce(i.uniqueFacets, [])), symbol: coalesce(([]+i.symbol)[0], \'\')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets, [])), symbol: coalesce(([]+channel.symbol)[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets, [])), symbol: coalesce(([]+technique.symbol)[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets, [])), symbol: coalesce(([]+template.symbol)[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), unique_facets: apoc.coll.sort(coalesce(template_anat.uniqueFacets, [])), symbol: coalesce(([]+template_anat.symbol)[0], \'\')} ,image_folder: COALESCE(([]+irw.folder)[0], \'\'), image_nrrd: COALESCE(([]+irw.nrrd)[0], \'\'), image_thumbnail: COALESCE(([]+irw.thumbnail)[0], \'\'), image_swc: COALESCE(([]+irw.swc)[0], \'\'), image_obj: COALESCE(([]+irw.obj)[0], \'\'), image_wlz: COALESCE(([]+irw.wlz)[0], \'\'), index: coalesce(apoc.convert.toInteger(([]+irw.index)[0]), []) + [] }} }) END AS anatomy_channel_image ,primary,parents,relationships,related_individuals,xrefs OPTIONAL MATCH (primary)-[rp:has_reference]->(p:pub) where rp.typ = \'syn\' WITH CASE WHEN p is null THEN [] ELSE collect({ pub: { core: { short_form: p.short_form, label: coalesce(p.label,\'\'), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], \'\')} , PubMed: coalesce(([]+p.PMID)[0], \'\'), FlyBase: coalesce(([]+p.FlyBase)[0], \'\'), DOI: coalesce(([]+p.DOI)[0], \'\') } , synonym: { label: coalesce(([]+rp.value)[0], \'\'), scope: coalesce(rp.scope, \'\'), type: coalesce(([]+rp.has_synonym_type)[0],\'\') } }) END AS pub_syn,primary,parents,relationships,related_individuals,xrefs,anatomy_channel_image OPTIONAL MATCH (primary)-[rp:has_reference]->(p:pub) WHERE rp.typ = \'def\' WITH CASE WHEN p is null THEN [] ELSE collect({ core: { short_form: p.short_form, label: coalesce(p.label,\'\'), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], \'\')} , PubMed: coalesce(([]+p.PMID)[0], \'\'), FlyBase: coalesce(([]+p.FlyBase)[0], \'\'), DOI: coalesce(([]+p.DOI)[0], \'\') } ) END AS def_pubs,primary,parents,relationships,related_individuals,xrefs,anatomy_channel_image,pub_syn RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, \'Get JSON for Class\' AS query, \'c58c844\' AS version , parents, relationships, related_individuals, xrefs, anatomy_channel_image, pub_syn, def_pubs", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Get JSON for Neuron Class
Description: Get JSON for Neuron Class

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (primary:Class) WHERE primary.short_form in [$id] WITH primary OPTIONAL MATCH (o:Class)<-[r:SUBCLASSOF|INSTANCEOF]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], '')} ) END AS parents ,primary OPTIONAL MATCH (o:Class)<-[r {}]-(primary) WHERE (r.type='Related') AND ((not exists(r.hide_in_terminfo)) OR (not (r.hide_in_terminfo[0] = true))) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r), database_cross_reference: coalesce(r.database_cross_reference, []), confidence_value: coalesce(toString(r.confidence_value[0]), '') }, object: { short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], '')} }) END AS relationships ,primary,parents OPTIONAL MATCH (o)<-[r:term_replaced_by]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r), database_cross_reference: coalesce(r.database_cross_reference, []), confidence_value: coalesce(toString(r.confidence_value[0]), '') }, object: { short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], '')} }) END AS related_individuals ,primary,parents,relationships OPTIONAL MATCH (s:Site { short_form: apoc.convert.toList(primary.self_xref)[0] }) WITH CASE WHEN s IS NULL THEN [] ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], ''), accession: coalesce(primary.short_form, ''), link_text: primary.label + ' on ' + s.label, homepage: coalesce(s.homepage[0], ''), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,''), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], '')} , icon: coalesce(s.link_icon_url[0], ''), link_postfix: coalesce(s.link_postfix[0], '')}) END AS self_xref, primary, parents, relationships, related_individuals OPTIONAL MATCH (s:Site)<-[dbx:database_cross_reference]-(primary) WITH CASE WHEN s IS NULL THEN self_xref ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], ''), accession: coalesce(([]+dbx.accession)[0], ''), link_text: primary.label + ' on ' + s.label, homepage: coalesce(s.homepage[0], ''), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,''), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], '')} , icon: coalesce(s.link_icon_url[0], ''), link_postfix: coalesce(s.link_postfix[0], '')}) + self_xref END AS xrefs,primary,parents,relationships,related_individuals CALL apoc.cypher.run('WITH primary OPTIONAL MATCH (primary)<- [:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual)<-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 10', {primary:primary}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, primary, parents, relationships, related_individuals, xrefs OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,''), iri: i.iri, types: labels(i), unique_facets: apoc.coll.sort(coalesce(i.uniqueFacets, [])), symbol: coalesce(([]+i.symbol)[0], '')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets, [])), symbol: coalesce(([]+channel.symbol)[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets, [])), symbol: coalesce(([]+technique.symbol)[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets, [])), symbol: coalesce(([]+template.symbol)[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), unique_facets: apoc.coll.sort(coalesce(template_anat.uniqueFacets, [])), symbol: coalesce(([]+template_anat.symbol)[0], '')} ,image_folder: COALESCE(([]+irw.folder)[0], ''), image_nrrd: COALESCE(([]+irw.nrrd)[0], ''), image_thumbnail: COALESCE(([]+irw.thumbnail)[0], ''), image_swc: COALESCE(([]+irw.swc)[0], ''), image_obj: COALESCE(([]+irw.obj)[0], ''), image_wlz: COALESCE(([]+irw.wlz)[0], ''), index: coalesce(apoc.convert.toInteger(([]+irw.index)[0]), []) + [] }} }) END AS anatomy_channel_image ,primary,parents,relationships,related_individuals,xrefs OPTIONAL MATCH (primary)-[rp:has_reference]->(p:pub) where rp.typ = 'syn' WITH CASE WHEN p is null THEN [] ELSE collect({ pub: { core: { short_form: p.short_form, label: coalesce(p.label,''), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], '')} , PubMed: coalesce(([]+p.PMID)[0], ''), FlyBase: coalesce(([]+p.FlyBase)[0], ''), DOI: coalesce(([]+p.DOI)[0], '') } , synonym: { label: coalesce(([]+rp.value)[0], ''), scope: coalesce(rp.scope, ''), type: coalesce(([]+rp.has_synonym_type)[0],'') } }) END AS pub_syn,primary,parents,relationships,related_individuals,xrefs,anatomy_channel_image OPTIONAL MATCH (primary)-[rp:has_reference]->(p:pub) WHERE rp.typ = 'def' WITH CASE WHEN p is null THEN [] ELSE collect({ core: { short_form: p.short_form, label: coalesce(p.label,''), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], '')} , PubMed: coalesce(([]+p.PMID)[0], ''), FlyBase: coalesce(([]+p.FlyBase)[0], ''), DOI: coalesce(([]+p.DOI)[0], '') } ) END AS def_pubs,primary,parents,relationships,related_individuals,xrefs,anatomy_channel_image,pub_syn OPTIONAL MATCH (:Class { label: 'intersectional expression pattern'})<-[:SUBCLASSOF]-(ep:Class)<-[ar:part_of]-(anoni:Individual)-[:INSTANCEOF]->(primary) WITH CASE WHEN ep IS NULL THEN [] ELSE COLLECT({ short_form: ep.short_form, label: coalesce(ep.label,''), iri: ep.iri, types: labels(ep), unique_facets: apoc.coll.sort(coalesce(ep.uniqueFacets, [])), symbol: coalesce(([]+ep.symbol)[0], '')} ) END AS targeting_splits,primary,parents,relationships,related_individuals,xrefs,anatomy_channel_image,pub_syn,def_pubs RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, 'Get JSON for Neuron Class' AS query, 'c58c844' AS version , parents, relationships, related_individuals, xrefs, anatomy_channel_image, pub_syn, def_pubs, targeting_splits", "parameters" : { "id" : "id" }}
query_template = {'name': 'Get JSON for Neuron Class', 'description': 'Get JSON for Neuron Class', 'query': '"statement": "MATCH (primary:Class) WHERE primary.short_form in [$id] WITH primary OPTIONAL MATCH (o:Class)<-[r:SUBCLASSOF|INSTANCEOF]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], \'\')} ) END AS parents ,primary OPTIONAL MATCH (o:Class)<-[r {}]-(primary) WHERE (r.type=\'Related\') AND ((not exists(r.hide_in_terminfo)) OR (not (r.hide_in_terminfo[0] = true))) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r), database_cross_reference: coalesce(r.database_cross_reference, []), confidence_value: coalesce(toString(r.confidence_value[0]), \'\') }, object: { short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], \'\')} }) END AS relationships ,primary,parents OPTIONAL MATCH (o)<-[r:term_replaced_by]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r), database_cross_reference: coalesce(r.database_cross_reference, []), confidence_value: coalesce(toString(r.confidence_value[0]), \'\') }, object: { short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], \'\')} }) END AS related_individuals ,primary,parents,relationships OPTIONAL MATCH (s:Site { short_form: apoc.convert.toList(primary.self_xref)[0] }) WITH CASE WHEN s IS NULL THEN [] ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], \'\'), accession: coalesce(primary.short_form, \'\'), link_text: primary.label + \' on \' + s.label, homepage: coalesce(s.homepage[0], \'\'), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,\'\'), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], \'\')} , icon: coalesce(s.link_icon_url[0], \'\'), link_postfix: coalesce(s.link_postfix[0], \'\')}) END AS self_xref, primary, parents, relationships, related_individuals OPTIONAL MATCH (s:Site)<-[dbx:database_cross_reference]-(primary) WITH CASE WHEN s IS NULL THEN self_xref ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], \'\'), accession: coalesce(([]+dbx.accession)[0], \'\'), link_text: primary.label + \' on \' + s.label, homepage: coalesce(s.homepage[0], \'\'), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,\'\'), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], \'\')} , icon: coalesce(s.link_icon_url[0], \'\'), link_postfix: coalesce(s.link_postfix[0], \'\')}) + self_xref END AS xrefs,primary,parents,relationships,related_individuals CALL apoc.cypher.run(\'WITH primary OPTIONAL MATCH (primary)<- [:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual)<-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 10\', {primary:primary}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, primary, parents, relationships, related_individuals, xrefs OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,\'\'), iri: i.iri, types: labels(i), unique_facets: apoc.coll.sort(coalesce(i.uniqueFacets, [])), symbol: coalesce(([]+i.symbol)[0], \'\')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets, [])), symbol: coalesce(([]+channel.symbol)[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets, [])), symbol: coalesce(([]+technique.symbol)[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets, [])), symbol: coalesce(([]+template.symbol)[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), unique_facets: apoc.coll.sort(coalesce(template_anat.uniqueFacets, [])), symbol: coalesce(([]+template_anat.symbol)[0], \'\')} ,image_folder: COALESCE(([]+irw.folder)[0], \'\'), image_nrrd: COALESCE(([]+irw.nrrd)[0], \'\'), image_thumbnail: COALESCE(([]+irw.thumbnail)[0], \'\'), image_swc: COALESCE(([]+irw.swc)[0], \'\'), image_obj: COALESCE(([]+irw.obj)[0], \'\'), image_wlz: COALESCE(([]+irw.wlz)[0], \'\'), index: coalesce(apoc.convert.toInteger(([]+irw.index)[0]), []) + [] }} }) END AS anatomy_channel_image ,primary,parents,relationships,related_individuals,xrefs OPTIONAL MATCH (primary)-[rp:has_reference]->(p:pub) where rp.typ = \'syn\' WITH CASE WHEN p is null THEN [] ELSE collect({ pub: { core: { short_form: p.short_form, label: coalesce(p.label,\'\'), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], \'\')} , PubMed: coalesce(([]+p.PMID)[0], \'\'), FlyBase: coalesce(([]+p.FlyBase)[0], \'\'), DOI: coalesce(([]+p.DOI)[0], \'\') } , synonym: { label: coalesce(([]+rp.value)[0], \'\'), scope: coalesce(rp.scope, \'\'), type: coalesce(([]+rp.has_synonym_type)[0],\'\') } }) END AS pub_syn,primary,parents,relationships,related_individuals,xrefs,anatomy_channel_image OPTIONAL MATCH (primary)-[rp:has_reference]->(p:pub) WHERE rp.typ = \'def\' WITH CASE WHEN p is null THEN [] ELSE collect({ core: { short_form: p.short_form, label: coalesce(p.label,\'\'), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], \'\')} , PubMed: coalesce(([]+p.PMID)[0], \'\'), FlyBase: coalesce(([]+p.FlyBase)[0], \'\'), DOI: coalesce(([]+p.DOI)[0], \'\') } ) END AS def_pubs,primary,parents,relationships,related_individuals,xrefs,anatomy_channel_image,pub_syn OPTIONAL MATCH (:Class { label: \'intersectional expression pattern\'})<-[:SUBCLASSOF]-(ep:Class)<-[ar:part_of]-(anoni:Individual)-[:INSTANCEOF]->(primary) WITH CASE WHEN ep IS NULL THEN [] ELSE COLLECT({ short_form: ep.short_form, label: coalesce(ep.label,\'\'), iri: ep.iri, types: labels(ep), unique_facets: apoc.coll.sort(coalesce(ep.uniqueFacets, [])), symbol: coalesce(([]+ep.symbol)[0], \'\')} ) END AS targeting_splits,primary,parents,relationships,related_individuals,xrefs,anatomy_channel_image,pub_syn,def_pubs RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, \'Get JSON for Neuron Class\' AS query, \'c58c844\' AS version , parents, relationships, related_individuals, xrefs, anatomy_channel_image, pub_syn, def_pubs, targeting_splits", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Get JSON for Split Class
Description: Get JSON for Split Class

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (primary:Class) WHERE primary.short_form in [$id] WITH primary OPTIONAL MATCH (o:Class)<-[r:SUBCLASSOF|INSTANCEOF]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], '')} ) END AS parents ,primary OPTIONAL MATCH (o:Class)<-[r {}]-(primary) WHERE (r.type='Related') AND ((not exists(r.hide_in_terminfo)) OR (not (r.hide_in_terminfo[0] = true))) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r), database_cross_reference: coalesce(r.database_cross_reference, []), confidence_value: coalesce(toString(r.confidence_value[0]), '') }, object: { short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], '')} }) END AS relationships ,primary,parents OPTIONAL MATCH (o)<-[r:term_replaced_by]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r), database_cross_reference: coalesce(r.database_cross_reference, []), confidence_value: coalesce(toString(r.confidence_value[0]), '') }, object: { short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], '')} }) END AS related_individuals ,primary,parents,relationships OPTIONAL MATCH (s:Site { short_form: apoc.convert.toList(primary.self_xref)[0] }) WITH CASE WHEN s IS NULL THEN [] ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], ''), accession: coalesce(primary.short_form, ''), link_text: primary.label + ' on ' + s.label, homepage: coalesce(s.homepage[0], ''), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,''), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], '')} , icon: coalesce(s.link_icon_url[0], ''), link_postfix: coalesce(s.link_postfix[0], '')}) END AS self_xref, primary, parents, relationships, related_individuals OPTIONAL MATCH (s:Site)<-[dbx:database_cross_reference]-(primary) WITH CASE WHEN s IS NULL THEN self_xref ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], ''), accession: coalesce(([]+dbx.accession)[0], ''), link_text: primary.label + ' on ' + s.label, homepage: coalesce(s.homepage[0], ''), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,''), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], '')} , icon: coalesce(s.link_icon_url[0], ''), link_postfix: coalesce(s.link_postfix[0], '')}) + self_xref END AS xrefs,primary,parents,relationships,related_individuals CALL apoc.cypher.run('WITH primary OPTIONAL MATCH (primary)<- [:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual)<-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 10', {primary:primary}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, primary, parents, relationships, related_individuals, xrefs OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,''), iri: i.iri, types: labels(i), unique_facets: apoc.coll.sort(coalesce(i.uniqueFacets, [])), symbol: coalesce(([]+i.symbol)[0], '')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets, [])), symbol: coalesce(([]+channel.symbol)[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets, [])), symbol: coalesce(([]+technique.symbol)[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets, [])), symbol: coalesce(([]+template.symbol)[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), unique_facets: apoc.coll.sort(coalesce(template_anat.uniqueFacets, [])), symbol: coalesce(([]+template_anat.symbol)[0], '')} ,image_folder: COALESCE(([]+irw.folder)[0], ''), image_nrrd: COALESCE(([]+irw.nrrd)[0], ''), image_thumbnail: COALESCE(([]+irw.thumbnail)[0], ''), image_swc: COALESCE(([]+irw.swc)[0], ''), image_obj: COALESCE(([]+irw.obj)[0], ''), image_wlz: COALESCE(([]+irw.wlz)[0], ''), index: coalesce(apoc.convert.toInteger(([]+irw.index)[0]), []) + [] }} }) END AS anatomy_channel_image ,primary,parents,relationships,related_individuals,xrefs OPTIONAL MATCH (primary)-[rp:has_reference]->(p:pub) where rp.typ = 'syn' WITH CASE WHEN p is null THEN [] ELSE collect({ pub: { core: { short_form: p.short_form, label: coalesce(p.label,''), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], '')} , PubMed: coalesce(([]+p.PMID)[0], ''), FlyBase: coalesce(([]+p.FlyBase)[0], ''), DOI: coalesce(([]+p.DOI)[0], '') } , synonym: { label: coalesce(([]+rp.value)[0], ''), scope: coalesce(rp.scope, ''), type: coalesce(([]+rp.has_synonym_type)[0],'') } }) END AS pub_syn,primary,parents,relationships,related_individuals,xrefs,anatomy_channel_image OPTIONAL MATCH (primary)-[rp:has_reference]->(p:pub) WHERE rp.typ = 'def' WITH CASE WHEN p is null THEN [] ELSE collect({ core: { short_form: p.short_form, label: coalesce(p.label,''), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], '')} , PubMed: coalesce(([]+p.PMID)[0], ''), FlyBase: coalesce(([]+p.FlyBase)[0], ''), DOI: coalesce(([]+p.DOI)[0], '') } ) END AS def_pubs,primary,parents,relationships,related_individuals,xrefs,anatomy_channel_image,pub_syn OPTIONAL MATCH (:Class { label: 'intersectional expression pattern'})<-[:SUBCLASSOF]-(primary)<-[ar:part_of]-(anoni:Individual)-[:INSTANCEOF]->(n:Neuron) WITH CASE WHEN n IS NULL THEN [] ELSE COLLECT({ short_form: n.short_form, label: coalesce(n.label,''), iri: n.iri, types: labels(n), unique_facets: apoc.coll.sort(coalesce(n.uniqueFacets, [])), symbol: coalesce(([]+n.symbol)[0], '')} ) END AS target_neurons,primary,parents,relationships,related_individuals,xrefs,anatomy_channel_image,pub_syn,def_pubs RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, 'Get JSON for Split Class' AS query, 'c58c844' AS version , parents, relationships, related_individuals, xrefs, anatomy_channel_image, pub_syn, def_pubs, target_neurons", "parameters" : { "id" : "id" }}
query_template = {'name': 'Get JSON for Split Class', 'description': 'Get JSON for Split Class', 'query': '"statement": "MATCH (primary:Class) WHERE primary.short_form in [$id] WITH primary OPTIONAL MATCH (o:Class)<-[r:SUBCLASSOF|INSTANCEOF]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], \'\')} ) END AS parents ,primary OPTIONAL MATCH (o:Class)<-[r {}]-(primary) WHERE (r.type=\'Related\') AND ((not exists(r.hide_in_terminfo)) OR (not (r.hide_in_terminfo[0] = true))) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r), database_cross_reference: coalesce(r.database_cross_reference, []), confidence_value: coalesce(toString(r.confidence_value[0]), \'\') }, object: { short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], \'\')} }) END AS relationships ,primary,parents OPTIONAL MATCH (o)<-[r:term_replaced_by]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r), database_cross_reference: coalesce(r.database_cross_reference, []), confidence_value: coalesce(toString(r.confidence_value[0]), \'\') }, object: { short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], \'\')} }) END AS related_individuals ,primary,parents,relationships OPTIONAL MATCH (s:Site { short_form: apoc.convert.toList(primary.self_xref)[0] }) WITH CASE WHEN s IS NULL THEN [] ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], \'\'), accession: coalesce(primary.short_form, \'\'), link_text: primary.label + \' on \' + s.label, homepage: coalesce(s.homepage[0], \'\'), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,\'\'), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], \'\')} , icon: coalesce(s.link_icon_url[0], \'\'), link_postfix: coalesce(s.link_postfix[0], \'\')}) END AS self_xref, primary, parents, relationships, related_individuals OPTIONAL MATCH (s:Site)<-[dbx:database_cross_reference]-(primary) WITH CASE WHEN s IS NULL THEN self_xref ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], \'\'), accession: coalesce(([]+dbx.accession)[0], \'\'), link_text: primary.label + \' on \' + s.label, homepage: coalesce(s.homepage[0], \'\'), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,\'\'), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], \'\')} , icon: coalesce(s.link_icon_url[0], \'\'), link_postfix: coalesce(s.link_postfix[0], \'\')}) + self_xref END AS xrefs,primary,parents,relationships,related_individuals CALL apoc.cypher.run(\'WITH primary OPTIONAL MATCH (primary)<- [:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual)<-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 10\', {primary:primary}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, primary, parents, relationships, related_individuals, xrefs OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,\'\'), iri: i.iri, types: labels(i), unique_facets: apoc.coll.sort(coalesce(i.uniqueFacets, [])), symbol: coalesce(([]+i.symbol)[0], \'\')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets, [])), symbol: coalesce(([]+channel.symbol)[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets, [])), symbol: coalesce(([]+technique.symbol)[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets, [])), symbol: coalesce(([]+template.symbol)[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), unique_facets: apoc.coll.sort(coalesce(template_anat.uniqueFacets, [])), symbol: coalesce(([]+template_anat.symbol)[0], \'\')} ,image_folder: COALESCE(([]+irw.folder)[0], \'\'), image_nrrd: COALESCE(([]+irw.nrrd)[0], \'\'), image_thumbnail: COALESCE(([]+irw.thumbnail)[0], \'\'), image_swc: COALESCE(([]+irw.swc)[0], \'\'), image_obj: COALESCE(([]+irw.obj)[0], \'\'), image_wlz: COALESCE(([]+irw.wlz)[0], \'\'), index: coalesce(apoc.convert.toInteger(([]+irw.index)[0]), []) + [] }} }) END AS anatomy_channel_image ,primary,parents,relationships,related_individuals,xrefs OPTIONAL MATCH (primary)-[rp:has_reference]->(p:pub) where rp.typ = \'syn\' WITH CASE WHEN p is null THEN [] ELSE collect({ pub: { core: { short_form: p.short_form, label: coalesce(p.label,\'\'), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], \'\')} , PubMed: coalesce(([]+p.PMID)[0], \'\'), FlyBase: coalesce(([]+p.FlyBase)[0], \'\'), DOI: coalesce(([]+p.DOI)[0], \'\') } , synonym: { label: coalesce(([]+rp.value)[0], \'\'), scope: coalesce(rp.scope, \'\'), type: coalesce(([]+rp.has_synonym_type)[0],\'\') } }) END AS pub_syn,primary,parents,relationships,related_individuals,xrefs,anatomy_channel_image OPTIONAL MATCH (primary)-[rp:has_reference]->(p:pub) WHERE rp.typ = \'def\' WITH CASE WHEN p is null THEN [] ELSE collect({ core: { short_form: p.short_form, label: coalesce(p.label,\'\'), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], \'\')} , PubMed: coalesce(([]+p.PMID)[0], \'\'), FlyBase: coalesce(([]+p.FlyBase)[0], \'\'), DOI: coalesce(([]+p.DOI)[0], \'\') } ) END AS def_pubs,primary,parents,relationships,related_individuals,xrefs,anatomy_channel_image,pub_syn OPTIONAL MATCH (:Class { label: \'intersectional expression pattern\'})<-[:SUBCLASSOF]-(primary)<-[ar:part_of]-(anoni:Individual)-[:INSTANCEOF]->(n:Neuron) WITH CASE WHEN n IS NULL THEN [] ELSE COLLECT({ short_form: n.short_form, label: coalesce(n.label,\'\'), iri: n.iri, types: labels(n), unique_facets: apoc.coll.sort(coalesce(n.uniqueFacets, [])), symbol: coalesce(([]+n.symbol)[0], \'\')} ) END AS target_neurons,primary,parents,relationships,related_individuals,xrefs,anatomy_channel_image,pub_syn,def_pubs RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, \'Get JSON for Split Class\' AS query, \'c58c844\' AS version , parents, relationships, related_individuals, xrefs, anatomy_channel_image, pub_syn, def_pubs, target_neurons", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Get JSON for Individual
Description: Get JSON for Individual:Anatomy

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (primary:Individual) WHERE primary.short_form in [$id] WITH primary OPTIONAL MATCH (primary)-[:has_source]-(ds:DataSet)-[:has_license|license]->(l:License) WITH COLLECT ({ dataset: { link : coalesce(([]+ds.dataset_link)[0], ''), core : { short_form: ds.short_form, label: coalesce(ds.label,''), iri: ds.iri, types: labels(ds), unique_facets: apoc.coll.sort(coalesce(ds.uniqueFacets, [])), symbol: coalesce(([]+ds.symbol)[0], '')} }, license: { icon : coalesce(([]+l.license_logo)[0], ''), link : coalesce(([]+l.license_url)[0], ''), core : { short_form: l.short_form, label: coalesce(l.label,''), iri: l.iri, types: labels(l), unique_facets: apoc.coll.sort(coalesce(l.uniqueFacets, [])), symbol: coalesce(([]+l.symbol)[0], '')} }}) AS dataset_license,primary OPTIONAL MATCH (o:Class)<-[r:SUBCLASSOF|INSTANCEOF]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], '')} ) END AS parents ,primary,dataset_license OPTIONAL MATCH (o:Class)<-[r {}]-(primary) WHERE (r.type='Related') AND ((not exists(r.hide_in_terminfo)) OR (not (r.hide_in_terminfo[0] = true))) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r), database_cross_reference: coalesce(r.database_cross_reference, []), confidence_value: coalesce(toString(r.confidence_value[0]), '') }, object: { short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], '')} }) END AS relationships ,primary,dataset_license,parents OPTIONAL MATCH (o)<-[r:term_replaced_by]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r), database_cross_reference: coalesce(r.database_cross_reference, []), confidence_value: coalesce(toString(r.confidence_value[0]), '') }, object: { short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], '')} }) END AS related_individuals ,primary,dataset_license,parents,relationships OPTIONAL MATCH (s:Site { short_form: apoc.convert.toList(primary.self_xref)[0] }) WITH CASE WHEN s IS NULL THEN [] ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], ''), accession: coalesce(primary.short_form, ''), link_text: primary.label + ' on ' + s.label, homepage: coalesce(s.homepage[0], ''), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,''), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], '')} , icon: coalesce(s.link_icon_url[0], ''), link_postfix: coalesce(s.link_postfix[0], '')}) END AS self_xref, primary, dataset_license, parents, relationships, related_individuals OPTIONAL MATCH (s:Site)<-[dbx:database_cross_reference]-(primary) WITH CASE WHEN s IS NULL THEN self_xref ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], ''), accession: coalesce(([]+dbx.accession)[0], ''), link_text: primary.label + ' on ' + s.label, homepage: coalesce(s.homepage[0], ''), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,''), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], '')} , icon: coalesce(s.link_icon_url[0], ''), link_postfix: coalesce(s.link_postfix[0], '')}) + self_xref END AS xrefs,primary,dataset_license,parents,relationships,related_individuals OPTIONAL MATCH (primary)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(template_anat:Individual) WITH template, channel, template_anat, irw, primary, dataset_license, parents, relationships, related_individuals, xrefs OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE collect ({ channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets, [])), symbol: coalesce(([]+channel.symbol)[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets, [])), symbol: coalesce(([]+technique.symbol)[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets, [])), symbol: coalesce(([]+template.symbol)[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), unique_facets: apoc.coll.sort(coalesce(template_anat.uniqueFacets, [])), symbol: coalesce(([]+template_anat.symbol)[0], '')} ,image_folder: COALESCE(([]+irw.folder)[0], ''), image_nrrd: COALESCE(([]+irw.nrrd)[0], ''), image_thumbnail: COALESCE(([]+irw.thumbnail)[0], ''), image_swc: COALESCE(([]+irw.swc)[0], ''), image_obj: COALESCE(([]+irw.obj)[0], ''), image_wlz: COALESCE(([]+irw.wlz)[0], ''), index: coalesce(apoc.convert.toInteger(([]+irw.index)[0]), []) + [] }}) END AS channel_image,primary,dataset_license,parents,relationships,related_individuals,xrefs OPTIONAL MATCH (primary)-[rp:has_reference]->(p:pub) where rp.typ = 'syn' WITH CASE WHEN p is null THEN [] ELSE collect({ pub: { core: { short_form: p.short_form, label: coalesce(p.label,''), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], '')} , PubMed: coalesce(([]+p.PMID)[0], ''), FlyBase: coalesce(([]+p.FlyBase)[0], ''), DOI: coalesce(([]+p.DOI)[0], '') } , synonym: { label: coalesce(([]+rp.value)[0], ''), scope: coalesce(rp.scope, ''), type: coalesce(([]+rp.has_synonym_type)[0],'') } }) END AS pub_syn,primary,dataset_license,parents,relationships,related_individuals,xrefs,channel_image OPTIONAL MATCH (primary)-[rp:has_reference]->(p:pub) WHERE rp.typ = 'def' WITH CASE WHEN p is null THEN [] ELSE collect({ core: { short_form: p.short_form, label: coalesce(p.label,''), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], '')} , PubMed: coalesce(([]+p.PMID)[0], ''), FlyBase: coalesce(([]+p.FlyBase)[0], ''), DOI: coalesce(([]+p.DOI)[0], '') } ) END AS def_pubs,primary,dataset_license,parents,relationships,related_individuals,xrefs,channel_image,pub_syn RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, 'Get JSON for Individual' AS query, 'c58c844' AS version , dataset_license, parents, relationships, related_individuals, xrefs, channel_image, pub_syn, def_pubs", "parameters" : { "id" : "id" }}
query_template = {'name': 'Get JSON for Individual', 'description': 'Get JSON for Individual:Anatomy', 'query': '"statement": "MATCH (primary:Individual) WHERE primary.short_form in [$id] WITH primary OPTIONAL MATCH (primary)-[:has_source]-(ds:DataSet)-[:has_license|license]->(l:License) WITH COLLECT ({ dataset: { link : coalesce(([]+ds.dataset_link)[0], \'\'), core : { short_form: ds.short_form, label: coalesce(ds.label,\'\'), iri: ds.iri, types: labels(ds), unique_facets: apoc.coll.sort(coalesce(ds.uniqueFacets, [])), symbol: coalesce(([]+ds.symbol)[0], \'\')} }, license: { icon : coalesce(([]+l.license_logo)[0], \'\'), link : coalesce(([]+l.license_url)[0], \'\'), core : { short_form: l.short_form, label: coalesce(l.label,\'\'), iri: l.iri, types: labels(l), unique_facets: apoc.coll.sort(coalesce(l.uniqueFacets, [])), symbol: coalesce(([]+l.symbol)[0], \'\')} }}) AS dataset_license,primary OPTIONAL MATCH (o:Class)<-[r:SUBCLASSOF|INSTANCEOF]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], \'\')} ) END AS parents ,primary,dataset_license OPTIONAL MATCH (o:Class)<-[r {}]-(primary) WHERE (r.type=\'Related\') AND ((not exists(r.hide_in_terminfo)) OR (not (r.hide_in_terminfo[0] = true))) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r), database_cross_reference: coalesce(r.database_cross_reference, []), confidence_value: coalesce(toString(r.confidence_value[0]), \'\') }, object: { short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], \'\')} }) END AS relationships ,primary,dataset_license,parents OPTIONAL MATCH (o)<-[r:term_replaced_by]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r), database_cross_reference: coalesce(r.database_cross_reference, []), confidence_value: coalesce(toString(r.confidence_value[0]), \'\') }, object: { short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], \'\')} }) END AS related_individuals ,primary,dataset_license,parents,relationships OPTIONAL MATCH (s:Site { short_form: apoc.convert.toList(primary.self_xref)[0] }) WITH CASE WHEN s IS NULL THEN [] ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], \'\'), accession: coalesce(primary.short_form, \'\'), link_text: primary.label + \' on \' + s.label, homepage: coalesce(s.homepage[0], \'\'), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,\'\'), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], \'\')} , icon: coalesce(s.link_icon_url[0], \'\'), link_postfix: coalesce(s.link_postfix[0], \'\')}) END AS self_xref, primary, dataset_license, parents, relationships, related_individuals OPTIONAL MATCH (s:Site)<-[dbx:database_cross_reference]-(primary) WITH CASE WHEN s IS NULL THEN self_xref ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], \'\'), accession: coalesce(([]+dbx.accession)[0], \'\'), link_text: primary.label + \' on \' + s.label, homepage: coalesce(s.homepage[0], \'\'), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,\'\'), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], \'\')} , icon: coalesce(s.link_icon_url[0], \'\'), link_postfix: coalesce(s.link_postfix[0], \'\')}) + self_xref END AS xrefs,primary,dataset_license,parents,relationships,related_individuals OPTIONAL MATCH (primary)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(template_anat:Individual) WITH template, channel, template_anat, irw, primary, dataset_license, parents, relationships, related_individuals, xrefs OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE collect ({ channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets, [])), symbol: coalesce(([]+channel.symbol)[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets, [])), symbol: coalesce(([]+technique.symbol)[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets, [])), symbol: coalesce(([]+template.symbol)[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), unique_facets: apoc.coll.sort(coalesce(template_anat.uniqueFacets, [])), symbol: coalesce(([]+template_anat.symbol)[0], \'\')} ,image_folder: COALESCE(([]+irw.folder)[0], \'\'), image_nrrd: COALESCE(([]+irw.nrrd)[0], \'\'), image_thumbnail: COALESCE(([]+irw.thumbnail)[0], \'\'), image_swc: COALESCE(([]+irw.swc)[0], \'\'), image_obj: COALESCE(([]+irw.obj)[0], \'\'), image_wlz: COALESCE(([]+irw.wlz)[0], \'\'), index: coalesce(apoc.convert.toInteger(([]+irw.index)[0]), []) + [] }}) END AS channel_image,primary,dataset_license,parents,relationships,related_individuals,xrefs OPTIONAL MATCH (primary)-[rp:has_reference]->(p:pub) where rp.typ = \'syn\' WITH CASE WHEN p is null THEN [] ELSE collect({ pub: { core: { short_form: p.short_form, label: coalesce(p.label,\'\'), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], \'\')} , PubMed: coalesce(([]+p.PMID)[0], \'\'), FlyBase: coalesce(([]+p.FlyBase)[0], \'\'), DOI: coalesce(([]+p.DOI)[0], \'\') } , synonym: { label: coalesce(([]+rp.value)[0], \'\'), scope: coalesce(rp.scope, \'\'), type: coalesce(([]+rp.has_synonym_type)[0],\'\') } }) END AS pub_syn,primary,dataset_license,parents,relationships,related_individuals,xrefs,channel_image OPTIONAL MATCH (primary)-[rp:has_reference]->(p:pub) WHERE rp.typ = \'def\' WITH CASE WHEN p is null THEN [] ELSE collect({ core: { short_form: p.short_form, label: coalesce(p.label,\'\'), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], \'\')} , PubMed: coalesce(([]+p.PMID)[0], \'\'), FlyBase: coalesce(([]+p.FlyBase)[0], \'\'), DOI: coalesce(([]+p.DOI)[0], \'\') } ) END AS def_pubs,primary,dataset_license,parents,relationships,related_individuals,xrefs,channel_image,pub_syn RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, \'Get JSON for Individual\' AS query, \'c58c844\' AS version , dataset_license, parents, relationships, related_individuals, xrefs, channel_image, pub_syn, def_pubs", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Get JSON for Cluster
Description: Get JSON for Cluster

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (primary:Cluster) WHERE primary.short_form in [$id] WITH primary OPTIONAL MATCH (primary)-[:has_source]-(ds:DataSet)-[:has_license|license]->(l:License) WITH COLLECT ({ dataset: { link : coalesce(ds.dataset_link[0], ''), core : { short_form: ds.short_form, label: coalesce(ds.label,''), iri: ds.iri, types: labels(ds), unique_facets: apoc.coll.sort(coalesce(ds.uniqueFacets, [])), symbol: coalesce(ds.symbol[0], '')} }, license: { icon : coalesce(l.license_logo[0], ''), link : coalesce(l.license_url[0], ''), core : { short_form: l.short_form, label: coalesce(l.label,''), iri: l.iri, types: labels(l), unique_facets: apoc.coll.sort(coalesce(l.uniqueFacets, [])), symbol: coalesce(l.symbol[0], '')} }}) AS dataset_license,primary OPTIONAL MATCH (o:Class)<-[r:SUBCLASSOF|INSTANCEOF]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(o.symbol[0], '')} ) END AS parents ,primary,dataset_license OPTIONAL MATCH (o:Class)<-[r {type:'Related'}]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r) } , object: { short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(o.symbol[0], '')} }) END AS relationships ,primary,dataset_license,parents OPTIONAL MATCH (s:Site { short_form: apoc.convert.toList(primary.self_xref)[0] }) WITH CASE WHEN s IS NULL THEN [] ELSE COLLECT({ link_base: coalesce(s.link_base[0], ''), accession: coalesce(primary.short_form, ''), link_text: primary.label + ' on ' + s.label, homepage: coalesce(s.homepage[0], ''), site: { short_form: s.short_form, label: coalesce(s.label,''), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(s.symbol[0], '')} , icon: coalesce(s.link_icon_url[0], ''), link_postfix: coalesce(s.link_postfix[0], '')}) END AS self_xref, primary, dataset_license, parents, relationships OPTIONAL MATCH (s:Site)<-[dbx:database_cross_reference]-(primary) WITH CASE WHEN s IS NULL THEN self_xref ELSE COLLECT({ link_base: coalesce(s.link_base[0], ''), accession: coalesce(dbx.accession[0], ''), link_text: primary.label + ' on ' + s.label, homepage: coalesce(s.homepage[0], ''), site: { short_form: s.short_form, label: coalesce(s.label,''), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(s.symbol[0], '')} , icon: coalesce(s.link_icon_url[0], ''), link_postfix: coalesce(s.link_postfix[0], '')}) + self_xref END AS xrefs,primary,dataset_license,parents,relationships OPTIONAL MATCH (primary)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(template_anat:Individual) WITH template, channel, template_anat, irw, primary, dataset_license, parents, relationships, xrefs OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE collect ({ channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets, [])), symbol: coalesce(channel.symbol[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets, [])), symbol: coalesce(technique.symbol[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets, [])), symbol: coalesce(template.symbol[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), unique_facets: apoc.coll.sort(coalesce(template_anat.uniqueFacets, [])), symbol: coalesce(template_anat.symbol[0], '')} ,image_folder: COALESCE(irw.folder[0], ''), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }}) END AS channel_image,primary,dataset_license,parents,relationships,xrefs OPTIONAL MATCH (o:Individual)<-[r {type:'Related'}]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.uri, type: type(r) } , object: { short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(o.symbol[0], ''), types: labels(o) }  }) END AS related_individuals ,primary,dataset_license,parents,relationships,xrefs,channel_image RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary) , unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(primary.symbol[0], '')}, description: coalesce(primary.description, []), comment : coalesce(primary.comment, primary.`annotation-comment`, []) } AS term, 'Get JSON for Cluster using modified Individual:Anatomy' AS query, 'ca9ab19' AS version , dataset_license, parents, relationships, xrefs, channel_image, related_individuals", "parameters" : { "id" : "id" }}
query_template = {'name': 'Get JSON for Cluster', 'description': 'Get JSON for Cluster', 'query': '"statement": "MATCH (primary:Cluster) WHERE primary.short_form in [$id] WITH primary OPTIONAL MATCH (primary)-[:has_source]-(ds:DataSet)-[:has_license|license]->(l:License) WITH COLLECT ({ dataset: { link : coalesce(ds.dataset_link[0], \'\'), core : { short_form: ds.short_form, label: coalesce(ds.label,\'\'), iri: ds.iri, types: labels(ds), unique_facets: apoc.coll.sort(coalesce(ds.uniqueFacets, [])), symbol: coalesce(ds.symbol[0], \'\')} }, license: { icon : coalesce(l.license_logo[0], \'\'), link : coalesce(l.license_url[0], \'\'), core : { short_form: l.short_form, label: coalesce(l.label,\'\'), iri: l.iri, types: labels(l), unique_facets: apoc.coll.sort(coalesce(l.uniqueFacets, [])), symbol: coalesce(l.symbol[0], \'\')} }}) AS dataset_license,primary OPTIONAL MATCH (o:Class)<-[r:SUBCLASSOF|INSTANCEOF]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(o.symbol[0], \'\')} ) END AS parents ,primary,dataset_license OPTIONAL MATCH (o:Class)<-[r {type:\'Related\'}]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r) } , object: { short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(o.symbol[0], \'\')} }) END AS relationships ,primary,dataset_license,parents OPTIONAL MATCH (s:Site { short_form: apoc.convert.toList(primary.self_xref)[0] }) WITH CASE WHEN s IS NULL THEN [] ELSE COLLECT({ link_base: coalesce(s.link_base[0], \'\'), accession: coalesce(primary.short_form, \'\'), link_text: primary.label + \' on \' + s.label, homepage: coalesce(s.homepage[0], \'\'), site: { short_form: s.short_form, label: coalesce(s.label,\'\'), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(s.symbol[0], \'\')} , icon: coalesce(s.link_icon_url[0], \'\'), link_postfix: coalesce(s.link_postfix[0], \'\')}) END AS self_xref, primary, dataset_license, parents, relationships OPTIONAL MATCH (s:Site)<-[dbx:database_cross_reference]-(primary) WITH CASE WHEN s IS NULL THEN self_xref ELSE COLLECT({ link_base: coalesce(s.link_base[0], \'\'), accession: coalesce(dbx.accession[0], \'\'), link_text: primary.label + \' on \' + s.label, homepage: coalesce(s.homepage[0], \'\'), site: { short_form: s.short_form, label: coalesce(s.label,\'\'), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(s.symbol[0], \'\')} , icon: coalesce(s.link_icon_url[0], \'\'), link_postfix: coalesce(s.link_postfix[0], \'\')}) + self_xref END AS xrefs,primary,dataset_license,parents,relationships OPTIONAL MATCH (primary)<-[:depicts]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(template_anat:Individual) WITH template, channel, template_anat, irw, primary, dataset_license, parents, relationships, xrefs OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE collect ({ channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets, [])), symbol: coalesce(channel.symbol[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets, [])), symbol: coalesce(technique.symbol[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets, [])), symbol: coalesce(template.symbol[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), unique_facets: apoc.coll.sort(coalesce(template_anat.uniqueFacets, [])), symbol: coalesce(template_anat.symbol[0], \'\')} ,image_folder: COALESCE(irw.folder[0], \'\'), index: coalesce(apoc.convert.toInteger(irw.index[0]), []) + [] }}) END AS channel_image,primary,dataset_license,parents,relationships,xrefs OPTIONAL MATCH (o:Individual)<-[r {type:\'Related\'}]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.uri, type: type(r) } , object: { short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(o.symbol[0], \'\'), types: labels(o) }  }) END AS related_individuals ,primary,dataset_license,parents,relationships,xrefs,channel_image RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary) , unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(primary.symbol[0], \'\')}, description: coalesce(primary.description, []), comment : coalesce(primary.comment, primary.`annotation-comment`, []) } AS term, \'Get JSON for Cluster using modified Individual:Anatomy\' AS query, \'ca9ab19\' AS version , dataset_license, parents, relationships, xrefs, channel_image, related_individuals", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Get JSON for Template
Description: Get JSON for Template

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (primary:Template) WHERE primary.short_form in [$id] WITH primary MATCH (channel:Individual)<-[irw:in_register_with]-(channel:Individual)-[:depicts]->(primary) WITH {index: coalesce(apoc.convert.toInteger(([]+irw.index)[0]), []) + [], extent: ([]+irw.extent)[0], center: ([]+irw.center)[0], voxel: ([]+irw.voxel)[0], orientation: coalesce(([]+irw.orientation)[0], ''), image_folder: coalesce(([]+irw.folder)[0],''), image_nrrd: COALESCE(([]+irw.nrrd)[0], ''), image_thumbnail: COALESCE(([]+irw.thumbnail)[0], ''), image_swc: COALESCE(([]+irw.swc)[0], ''), image_obj: COALESCE(([]+irw.obj)[0], ''), image_wlz: COALESCE(([]+irw.wlz)[0], ''), channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets, [])), symbol: coalesce(([]+channel.symbol)[0], '')} } as template_channel,primary OPTIONAL MATCH (technique:Class)<-[:is_specified_output_of]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(primary) WHERE technique.short_form IN ['FBbi_00000224','FBbi_00000251'] AND exists(irw.index) WITH primary, template_channel, collect ({ channel: channel, irw: irw}) AS painted_domains UNWIND painted_domains AS pd OPTIONAL MATCH (channel:Individual { short_form: pd.channel.short_form})-[:depicts]-(ai:Individual)-[:INSTANCEOF]->(c:Class) WITH collect({ anatomical_type: { short_form: c.short_form, label: coalesce(c.label,''), iri: c.iri, types: labels(c), unique_facets: apoc.coll.sort(coalesce(c.uniqueFacets, [])), symbol: coalesce(([]+c.symbol)[0], '')} , anatomical_individual: { short_form: ai.short_form, label: coalesce(ai.label,''), iri: ai.iri, types: labels(ai), unique_facets: apoc.coll.sort(coalesce(ai.uniqueFacets, [])), symbol: coalesce(([]+ai.symbol)[0], '')} , folder: COALESCE(([]+pd.irw.folder)[0], ''), image_nrrd: COALESCE(([]+pd.irw.nrrd)[0], ''), image_thumbnail: COALESCE(([]+pd.irw.thumbnail)[0], ''), image_swc: COALESCE(([]+pd.irw.swc)[0], ''), image_obj: COALESCE(([]+pd.irw.obj)[0], ''), image_wlz: COALESCE(([]+pd.irw.wlz)[0], ''), center: coalesce (pd.irw.center, []), index: coalesce(apoc.convert.toInteger(([]+pd.irw.index)[0]), []) + [] }) AS template_domains,primary,template_channel OPTIONAL MATCH (primary)-[:has_source]-(ds:DataSet)-[:has_license|license]->(l:License) WITH COLLECT ({ dataset: { link : coalesce(([]+ds.dataset_link)[0], ''), core : { short_form: ds.short_form, label: coalesce(ds.label,''), iri: ds.iri, types: labels(ds), unique_facets: apoc.coll.sort(coalesce(ds.uniqueFacets, [])), symbol: coalesce(([]+ds.symbol)[0], '')} }, license: { icon : coalesce(([]+l.license_logo)[0], ''), link : coalesce(([]+l.license_url)[0], ''), core : { short_form: l.short_form, label: coalesce(l.label,''), iri: l.iri, types: labels(l), unique_facets: apoc.coll.sort(coalesce(l.uniqueFacets, [])), symbol: coalesce(([]+l.symbol)[0], '')} }}) AS dataset_license,primary,template_channel,template_domains OPTIONAL MATCH (o:Class)<-[r:SUBCLASSOF|INSTANCEOF]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], '')} ) END AS parents ,primary,template_channel,template_domains,dataset_license OPTIONAL MATCH (o:Class)<-[r {}]-(primary) WHERE (r.type='Related') AND ((not exists(r.hide_in_terminfo)) OR (not (r.hide_in_terminfo[0] = true))) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r), database_cross_reference: coalesce(r.database_cross_reference, []), confidence_value: coalesce(toString(r.confidence_value[0]), '') }, object: { short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], '')} }) END AS relationships ,primary,template_channel,template_domains,dataset_license,parents OPTIONAL MATCH (s:Site { short_form: apoc.convert.toList(primary.self_xref)[0] }) WITH CASE WHEN s IS NULL THEN [] ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], ''), accession: coalesce(primary.short_form, ''), link_text: primary.label + ' on ' + s.label, homepage: coalesce(s.homepage[0], ''), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,''), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], '')} , icon: coalesce(s.link_icon_url[0], ''), link_postfix: coalesce(s.link_postfix[0], '')}) END AS self_xref, primary, template_channel, template_domains, dataset_license, parents, relationships OPTIONAL MATCH (s:Site)<-[dbx:database_cross_reference]-(primary) WITH CASE WHEN s IS NULL THEN self_xref ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], ''), accession: coalesce(([]+dbx.accession)[0], ''), link_text: primary.label + ' on ' + s.label, homepage: coalesce(s.homepage[0], ''), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,''), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], '')} , icon: coalesce(s.link_icon_url[0], ''), link_postfix: coalesce(s.link_postfix[0], '')}) + self_xref END AS xrefs,primary,template_channel,template_domains,dataset_license,parents,relationships OPTIONAL MATCH (o:Individual)<-[r]-(primary) WHERE (r.type='Related') OR r:term_replaced_by WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r), database_cross_reference: coalesce(r.database_cross_reference, []), confidence_value: coalesce(toString(r.confidence_value[0]), '') }, object: { short_form: o.short_form, label: coalesce(o.label,''), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], '')} }) END AS related_individuals ,primary,template_channel,template_domains,dataset_license,parents,relationships,xrefs RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, 'Get JSON for Template' AS query, 'c58c844' AS version , template_channel, template_domains, dataset_license, parents, relationships, xrefs, related_individuals", "parameters" : { "id" : "id" }}
query_template = {'name': 'Get JSON for Template', 'description': 'Get JSON for Template', 'query': '"statement": "MATCH (primary:Template) WHERE primary.short_form in [$id] WITH primary MATCH (channel:Individual)<-[irw:in_register_with]-(channel:Individual)-[:depicts]->(primary) WITH {index: coalesce(apoc.convert.toInteger(([]+irw.index)[0]), []) + [], extent: ([]+irw.extent)[0], center: ([]+irw.center)[0], voxel: ([]+irw.voxel)[0], orientation: coalesce(([]+irw.orientation)[0], \'\'), image_folder: coalesce(([]+irw.folder)[0],\'\'), image_nrrd: COALESCE(([]+irw.nrrd)[0], \'\'), image_thumbnail: COALESCE(([]+irw.thumbnail)[0], \'\'), image_swc: COALESCE(([]+irw.swc)[0], \'\'), image_obj: COALESCE(([]+irw.obj)[0], \'\'), image_wlz: COALESCE(([]+irw.wlz)[0], \'\'), channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets, [])), symbol: coalesce(([]+channel.symbol)[0], \'\')} } as template_channel,primary OPTIONAL MATCH (technique:Class)<-[:is_specified_output_of]-(channel:Individual)-[irw:in_register_with]->(template:Individual)-[:depicts]->(primary) WHERE technique.short_form IN [\'FBbi_00000224\',\'FBbi_00000251\'] AND exists(irw.index) WITH primary, template_channel, collect ({ channel: channel, irw: irw}) AS painted_domains UNWIND painted_domains AS pd OPTIONAL MATCH (channel:Individual { short_form: pd.channel.short_form})-[:depicts]-(ai:Individual)-[:INSTANCEOF]->(c:Class) WITH collect({ anatomical_type: { short_form: c.short_form, label: coalesce(c.label,\'\'), iri: c.iri, types: labels(c), unique_facets: apoc.coll.sort(coalesce(c.uniqueFacets, [])), symbol: coalesce(([]+c.symbol)[0], \'\')} , anatomical_individual: { short_form: ai.short_form, label: coalesce(ai.label,\'\'), iri: ai.iri, types: labels(ai), unique_facets: apoc.coll.sort(coalesce(ai.uniqueFacets, [])), symbol: coalesce(([]+ai.symbol)[0], \'\')} , folder: COALESCE(([]+pd.irw.folder)[0], \'\'), image_nrrd: COALESCE(([]+pd.irw.nrrd)[0], \'\'), image_thumbnail: COALESCE(([]+pd.irw.thumbnail)[0], \'\'), image_swc: COALESCE(([]+pd.irw.swc)[0], \'\'), image_obj: COALESCE(([]+pd.irw.obj)[0], \'\'), image_wlz: COALESCE(([]+pd.irw.wlz)[0], \'\'), center: coalesce (pd.irw.center, []), index: coalesce(apoc.convert.toInteger(([]+pd.irw.index)[0]), []) + [] }) AS template_domains,primary,template_channel OPTIONAL MATCH (primary)-[:has_source]-(ds:DataSet)-[:has_license|license]->(l:License) WITH COLLECT ({ dataset: { link : coalesce(([]+ds.dataset_link)[0], \'\'), core : { short_form: ds.short_form, label: coalesce(ds.label,\'\'), iri: ds.iri, types: labels(ds), unique_facets: apoc.coll.sort(coalesce(ds.uniqueFacets, [])), symbol: coalesce(([]+ds.symbol)[0], \'\')} }, license: { icon : coalesce(([]+l.license_logo)[0], \'\'), link : coalesce(([]+l.license_url)[0], \'\'), core : { short_form: l.short_form, label: coalesce(l.label,\'\'), iri: l.iri, types: labels(l), unique_facets: apoc.coll.sort(coalesce(l.uniqueFacets, [])), symbol: coalesce(([]+l.symbol)[0], \'\')} }}) AS dataset_license,primary,template_channel,template_domains OPTIONAL MATCH (o:Class)<-[r:SUBCLASSOF|INSTANCEOF]-(primary) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], \'\')} ) END AS parents ,primary,template_channel,template_domains,dataset_license OPTIONAL MATCH (o:Class)<-[r {}]-(primary) WHERE (r.type=\'Related\') AND ((not exists(r.hide_in_terminfo)) OR (not (r.hide_in_terminfo[0] = true))) WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r), database_cross_reference: coalesce(r.database_cross_reference, []), confidence_value: coalesce(toString(r.confidence_value[0]), \'\') }, object: { short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], \'\')} }) END AS relationships ,primary,template_channel,template_domains,dataset_license,parents OPTIONAL MATCH (s:Site { short_form: apoc.convert.toList(primary.self_xref)[0] }) WITH CASE WHEN s IS NULL THEN [] ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], \'\'), accession: coalesce(primary.short_form, \'\'), link_text: primary.label + \' on \' + s.label, homepage: coalesce(s.homepage[0], \'\'), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,\'\'), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], \'\')} , icon: coalesce(s.link_icon_url[0], \'\'), link_postfix: coalesce(s.link_postfix[0], \'\')}) END AS self_xref, primary, template_channel, template_domains, dataset_license, parents, relationships OPTIONAL MATCH (s:Site)<-[dbx:database_cross_reference]-(primary) WITH CASE WHEN s IS NULL THEN self_xref ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], \'\'), accession: coalesce(([]+dbx.accession)[0], \'\'), link_text: primary.label + \' on \' + s.label, homepage: coalesce(s.homepage[0], \'\'), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,\'\'), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], \'\')} , icon: coalesce(s.link_icon_url[0], \'\'), link_postfix: coalesce(s.link_postfix[0], \'\')}) + self_xref END AS xrefs,primary,template_channel,template_domains,dataset_license,parents,relationships OPTIONAL MATCH (o:Individual)<-[r]-(primary) WHERE (r.type=\'Related\') OR r:term_replaced_by WITH CASE WHEN o IS NULL THEN [] ELSE COLLECT ({ relation: { label: r.label, iri: r.iri, type: type(r), database_cross_reference: coalesce(r.database_cross_reference, []), confidence_value: coalesce(toString(r.confidence_value[0]), \'\') }, object: { short_form: o.short_form, label: coalesce(o.label,\'\'), iri: o.iri, types: labels(o), unique_facets: apoc.coll.sort(coalesce(o.uniqueFacets, [])), symbol: coalesce(([]+o.symbol)[0], \'\')} }) END AS related_individuals ,primary,template_channel,template_domains,dataset_license,parents,relationships,xrefs RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, \'Get JSON for Template\' AS query, \'c58c844\' AS version , template_channel, template_domains, dataset_license, parents, relationships, xrefs, related_individuals", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Get JSON for pub
Description: Fetches JSON for pub.

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (primary:Individual:pub) WHERE primary.short_form in [$id] WITH primary OPTIONAL MATCH (primary)-[:has_reference]-(ds:DataSet)-[:has_license|license]->(l:License) WITH COLLECT ({ dataset: { link : coalesce(([]+ds.dataset_link)[0], ''), core : { short_form: ds.short_form, label: coalesce(ds.label,''), iri: ds.iri, types: labels(ds), unique_facets: apoc.coll.sort(coalesce(ds.uniqueFacets, [])), symbol: coalesce(([]+ds.symbol)[0], '')} }, license: { icon : coalesce(([]+l.license_logo)[0], ''), link : coalesce(([]+l.license_url)[0], ''), core : { short_form: l.short_form, label: coalesce(l.label,''), iri: l.iri, types: labels(l), unique_facets: apoc.coll.sort(coalesce(l.uniqueFacets, [])), symbol: coalesce(([]+l.symbol)[0], '')} }}) AS dataset_license,primary RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, 'Get JSON for pub' AS query, 'c58c844' AS version , dataset_license, {title: coalesce(([]+primary.title)[0], '') ,PubMed: coalesce(([]+primary.PMID)[0], ''), FlyBase: coalesce(([]+primary.FlyBase)[0], ''), DOI: coalesce(([]+primary.DOI)[0], '') }AS pub_specific_content", "parameters" : { "id" : "id" }}
query_template = {'name': 'Get JSON for pub', 'description': 'Fetches JSON for pub.', 'query': '"statement": "MATCH (primary:Individual:pub) WHERE primary.short_form in [$id] WITH primary OPTIONAL MATCH (primary)-[:has_reference]-(ds:DataSet)-[:has_license|license]->(l:License) WITH COLLECT ({ dataset: { link : coalesce(([]+ds.dataset_link)[0], \'\'), core : { short_form: ds.short_form, label: coalesce(ds.label,\'\'), iri: ds.iri, types: labels(ds), unique_facets: apoc.coll.sort(coalesce(ds.uniqueFacets, [])), symbol: coalesce(([]+ds.symbol)[0], \'\')} }, license: { icon : coalesce(([]+l.license_logo)[0], \'\'), link : coalesce(([]+l.license_url)[0], \'\'), core : { short_form: l.short_form, label: coalesce(l.label,\'\'), iri: l.iri, types: labels(l), unique_facets: apoc.coll.sort(coalesce(l.uniqueFacets, [])), symbol: coalesce(([]+l.symbol)[0], \'\')} }}) AS dataset_license,primary RETURN { core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, \'Get JSON for pub\' AS query, \'c58c844\' AS version , dataset_license, {title: coalesce(([]+primary.title)[0], \'\') ,PubMed: coalesce(([]+primary.PMID)[0], \'\'), FlyBase: coalesce(([]+primary.FlyBase)[0], \'\'), DOI: coalesce(([]+primary.DOI)[0], \'\') }AS pub_specific_content", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Get JSON for DataSet
Description: Get JSON for DataSet

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (primary:DataSet) WHERE primary.short_form in [$id] WITH primary CALL apoc.cypher.run('WITH primary OPTIONAL MATCH (primary)<- [:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual)<-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 10', {primary:primary}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, primary OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,''), iri: i.iri, types: labels(i), unique_facets: apoc.coll.sort(coalesce(i.uniqueFacets, [])), symbol: coalesce(([]+i.symbol)[0], '')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,''), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets, [])), symbol: coalesce(([]+channel.symbol)[0], '')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,''), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets, [])), symbol: coalesce(([]+technique.symbol)[0], '')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,''), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets, [])), symbol: coalesce(([]+template.symbol)[0], '')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,''), iri: template_anat.iri, types: labels(template_anat), unique_facets: apoc.coll.sort(coalesce(template_anat.uniqueFacets, [])), symbol: coalesce(([]+template_anat.symbol)[0], '')} ,image_folder: COALESCE(([]+irw.folder)[0], ''), image_nrrd: COALESCE(([]+irw.nrrd)[0], ''), image_thumbnail: COALESCE(([]+irw.thumbnail)[0], ''), image_swc: COALESCE(([]+irw.swc)[0], ''), image_obj: COALESCE(([]+irw.obj)[0], ''), image_wlz: COALESCE(([]+irw.wlz)[0], ''), index: coalesce(apoc.convert.toInteger(([]+irw.index)[0]), []) + [] }} }) END AS anatomy_channel_image ,primary OPTIONAL MATCH (s:Site { short_form: apoc.convert.toList(primary.self_xref)[0] }) WITH CASE WHEN s IS NULL THEN [] ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], ''), accession: coalesce(primary.short_form, ''), link_text: primary.label + ' on ' + s.label, homepage: coalesce(s.homepage[0], ''), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,''), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], '')} , icon: coalesce(s.link_icon_url[0], ''), link_postfix: coalesce(s.link_postfix[0], '')}) END AS self_xref, primary, anatomy_channel_image OPTIONAL MATCH (s:Site)<-[dbx:database_cross_reference]-(primary) WITH CASE WHEN s IS NULL THEN self_xref ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], ''), accession: coalesce(([]+dbx.accession)[0], ''), link_text: primary.label + ' on ' + s.label, homepage: coalesce(s.homepage[0], ''), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,''), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], '')} , icon: coalesce(s.link_icon_url[0], ''), link_postfix: coalesce(s.link_postfix[0], '')}) + self_xref END AS xrefs,primary,anatomy_channel_image OPTIONAL MATCH (primary)-[:has_license|license]->(l:License) WITH collect ({ icon : coalesce(([]+l.license_logo)[0], ''), link : coalesce(([]+l.license_url)[0], ''), core : { short_form: l.short_form, label: coalesce(l.label,''), iri: l.iri, types: labels(l), unique_facets: apoc.coll.sort(coalesce(l.uniqueFacets, [])), symbol: coalesce(([]+l.symbol)[0], '')} }) as license,primary,anatomy_channel_image,xrefs OPTIONAL MATCH (primary)-[rp:has_reference]->(p:pub) WITH CASE WHEN p is null THEN [] ELSE collect({ core: { short_form: p.short_form, label: coalesce(p.label,''), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], '')} , PubMed: coalesce(([]+p.PMID)[0], ''), FlyBase: coalesce(([]+p.FlyBase)[0], ''), DOI: coalesce(([]+p.DOI)[0], '') } ) END AS pubs,primary,anatomy_channel_image,xrefs,license OPTIONAL MATCH (primary)<-[:has_source]-(i:Individual) WITH i, primary, anatomy_channel_image, xrefs, license, pubs OPTIONAL MATCH (i)-[:INSTANCEOF]-(c:Class) WITH DISTINCT { images: count(distinct i),types: count(distinct c) } as dataset_counts,primary,anatomy_channel_image,xrefs,license,pubs RETURN { link : coalesce(([]+primary.dataset_link)[0], ''), core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, 'Get JSON for DataSet' AS query, 'c58c844' AS version , anatomy_channel_image, xrefs, license, pubs, dataset_counts", "parameters" : { "id" : "id" }}
query_template = {'name': 'Get JSON for DataSet', 'description': 'Get JSON for DataSet', 'query': '"statement": "MATCH (primary:DataSet) WHERE primary.short_form in [$id] WITH primary CALL apoc.cypher.run(\'WITH primary OPTIONAL MATCH (primary)<- [:has_source|SUBCLASSOF|INSTANCEOF*]-(i:Individual)<-[:depicts]- (channel:Individual)-[irw:in_register_with] ->(template:Individual)-[:depicts]-> (template_anat:Individual) RETURN template, channel, template_anat, i, irw limit 10\', {primary:primary}) yield value with value.template as template, value.channel as channel,value.template_anat as template_anat, value.i as i, value.irw as irw, primary OPTIONAL MATCH (channel)-[:is_specified_output_of]->(technique:Class) WITH CASE WHEN channel IS NULL THEN [] ELSE COLLECT({ anatomy: { short_form: i.short_form, label: coalesce(i.label,\'\'), iri: i.iri, types: labels(i), unique_facets: apoc.coll.sort(coalesce(i.uniqueFacets, [])), symbol: coalesce(([]+i.symbol)[0], \'\')} , channel_image: { channel: { short_form: channel.short_form, label: coalesce(channel.label,\'\'), iri: channel.iri, types: labels(channel), unique_facets: apoc.coll.sort(coalesce(channel.uniqueFacets, [])), symbol: coalesce(([]+channel.symbol)[0], \'\')} , imaging_technique: { short_form: technique.short_form, label: coalesce(technique.label,\'\'), iri: technique.iri, types: labels(technique), unique_facets: apoc.coll.sort(coalesce(technique.uniqueFacets, [])), symbol: coalesce(([]+technique.symbol)[0], \'\')} ,image: { template_channel : { short_form: template.short_form, label: coalesce(template.label,\'\'), iri: template.iri, types: labels(template), unique_facets: apoc.coll.sort(coalesce(template.uniqueFacets, [])), symbol: coalesce(([]+template.symbol)[0], \'\')} , template_anatomy: { short_form: template_anat.short_form, label: coalesce(template_anat.label,\'\'), iri: template_anat.iri, types: labels(template_anat), unique_facets: apoc.coll.sort(coalesce(template_anat.uniqueFacets, [])), symbol: coalesce(([]+template_anat.symbol)[0], \'\')} ,image_folder: COALESCE(([]+irw.folder)[0], \'\'), image_nrrd: COALESCE(([]+irw.nrrd)[0], \'\'), image_thumbnail: COALESCE(([]+irw.thumbnail)[0], \'\'), image_swc: COALESCE(([]+irw.swc)[0], \'\'), image_obj: COALESCE(([]+irw.obj)[0], \'\'), image_wlz: COALESCE(([]+irw.wlz)[0], \'\'), index: coalesce(apoc.convert.toInteger(([]+irw.index)[0]), []) + [] }} }) END AS anatomy_channel_image ,primary OPTIONAL MATCH (s:Site { short_form: apoc.convert.toList(primary.self_xref)[0] }) WITH CASE WHEN s IS NULL THEN [] ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], \'\'), accession: coalesce(primary.short_form, \'\'), link_text: primary.label + \' on \' + s.label, homepage: coalesce(s.homepage[0], \'\'), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,\'\'), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], \'\')} , icon: coalesce(s.link_icon_url[0], \'\'), link_postfix: coalesce(s.link_postfix[0], \'\')}) END AS self_xref, primary, anatomy_channel_image OPTIONAL MATCH (s:Site)<-[dbx:database_cross_reference]-(primary) WITH CASE WHEN s IS NULL THEN self_xref ELSE COLLECT({ link_base: coalesce(([]+s.link_base)[0], \'\'), accession: coalesce(([]+dbx.accession)[0], \'\'), link_text: primary.label + \' on \' + s.label, homepage: coalesce(s.homepage[0], \'\'), is_data_source: coalesce(s.is_data_source[0], false), site: { short_form: s.short_form, label: coalesce(s.label,\'\'), iri: s.iri, types: labels(s), unique_facets: apoc.coll.sort(coalesce(s.uniqueFacets, [])), symbol: coalesce(([]+s.symbol)[0], \'\')} , icon: coalesce(s.link_icon_url[0], \'\'), link_postfix: coalesce(s.link_postfix[0], \'\')}) + self_xref END AS xrefs,primary,anatomy_channel_image OPTIONAL MATCH (primary)-[:has_license|license]->(l:License) WITH collect ({ icon : coalesce(([]+l.license_logo)[0], \'\'), link : coalesce(([]+l.license_url)[0], \'\'), core : { short_form: l.short_form, label: coalesce(l.label,\'\'), iri: l.iri, types: labels(l), unique_facets: apoc.coll.sort(coalesce(l.uniqueFacets, [])), symbol: coalesce(([]+l.symbol)[0], \'\')} }) as license,primary,anatomy_channel_image,xrefs OPTIONAL MATCH (primary)-[rp:has_reference]->(p:pub) WITH CASE WHEN p is null THEN [] ELSE collect({ core: { short_form: p.short_form, label: coalesce(p.label,\'\'), iri: p.iri, types: labels(p), unique_facets: apoc.coll.sort(coalesce(p.uniqueFacets, [])), symbol: coalesce(([]+p.symbol)[0], \'\')} , PubMed: coalesce(([]+p.PMID)[0], \'\'), FlyBase: coalesce(([]+p.FlyBase)[0], \'\'), DOI: coalesce(([]+p.DOI)[0], \'\') } ) END AS pubs,primary,anatomy_channel_image,xrefs,license OPTIONAL MATCH (primary)<-[:has_source]-(i:Individual) WITH i, primary, anatomy_channel_image, xrefs, license, pubs OPTIONAL MATCH (i)-[:INSTANCEOF]-(c:Class) WITH DISTINCT { images: count(distinct i),types: count(distinct c) } as dataset_counts,primary,anatomy_channel_image,xrefs,license,pubs RETURN { link : coalesce(([]+primary.dataset_link)[0], \'\'), core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, \'Get JSON for DataSet\' AS query, \'c58c844\' AS version , anatomy_channel_image, xrefs, license, pubs, dataset_counts", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Get JSON for License
Description: Get JSON for License

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (primary:License) WHERE primary.short_form in [$id] WITH primary RETURN { icon : coalesce(([]+primary.license_logo)[0], ''), link : coalesce(([]+primary.license_url)[0], ''), core : { short_form: primary.short_form, label: coalesce(primary.label,''), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], '')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, 'Get JSON for License' AS query, 'c58c844' AS version ", "parameters" : { "id" : "id" }}
query_template = {'name': 'Get JSON for License', 'description': 'Get JSON for License', 'query': '"statement": "MATCH (primary:License) WHERE primary.short_form in [$id] WITH primary RETURN { icon : coalesce(([]+primary.license_logo)[0], \'\'), link : coalesce(([]+primary.license_url)[0], \'\'), core : { short_form: primary.short_form, label: coalesce(primary.label,\'\'), iri: primary.iri, types: labels(primary), unique_facets: apoc.coll.sort(coalesce(primary.uniqueFacets, [])), symbol: coalesce(([]+primary.symbol)[0], \'\')} , description : coalesce(primary.description, []), comment : coalesce(primary.comment, []) } AS term, \'Get JSON for License\' AS query, \'c58c844\' AS version ", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Find images aligned to template id
Description: Find images aligned to template id

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (n:Template {short_form:$id})<-[:depicts]-(:Template)<-[r:in_register_with]-(dc:Individual)-[:depicts]->(di:Individual) RETURN COLLECT(distinct di.short_form) as ids", "parameters" : { "id" : "id" }}
query_template = {'name': 'Find images aligned to template id', 'description': 'Find images aligned to template id', 'query': '"statement": "MATCH (n:Template {short_form:$id})<-[:depicts]-(:Template)<-[r:in_register_with]-(dc:Individual)-[:depicts]->(di:Individual) RETURN COLLECT(distinct di.short_form) as ids", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## template_2_datasets_ids
Description: Get JSON for template_2_datasets ids

In [ ]:

# Insert test IDs here
id = 'YOUR_TEST_ID'
ids = ['ID1', 'ID2']

# Query
query = {"statement": "MATCH (t:Template) <-[depicts]-(tc:Template)-[:in_register_with]-(c:Individual)-[:depicts]->(ai:Individual)-[:has_source]->(ds:DataSet) WHERE t.short_form in [$id] RETURN distinct ds.short_form as ids", "parameters" : { "id" : "id" }}
query_template = {'name': 'template_2_datasets_ids', 'description': 'Get JSON for template_2_datasets ids', 'query': '"statement": "MATCH (t:Template) <-[depicts]-(tc:Template)-[:in_register_with]-(c:Individual)-[:depicts]->(ai:Individual)-[:has_source]->(ds:DataSet) WHERE t.short_form in [$id] RETURN distinct ds.short_form as ids", "parameters" : { "id" : "$ID" }', 'data_source_url': 'http://pdb.v4.virtualflybrain.org/db/neo4j/tx', 'data_source_type': 'neo4jDataSource'}

# Execute the query (example for Neo4j)
import requests
import time

start_time = time.time()
response = requests.post("http://pdb.v4.virtualflybrain.org/db/neo4j/tx", json={'statements': [query]})
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.json())
print('Time taken:', end_time - start_time)


## Owlery Part of
Description: Part of $NAME

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "http://owl.virtualflybrain.org/kbs/vfb/subclasses?" + "object=%3Chttp://purl.obolibrary.org/obo/BFO_0000050%3E%20some%20%3Chttp://purl.obolibrary.org/obo/{id}%3E&amp;direct=false&amp;includeDeprecated=false&amp;includeEquivalent=true".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)


## Owlery Neuron class with part here
Description: Neurons with some part here

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "http://owl.virtualflybrain.org/kbs/vfb/subclasses?" + "object=%3Chttp://purl.obolibrary.org/obo/FBbt_00005106%3E%20and%20%3Chttp://purl.obolibrary.org/obo/RO_0002131%3E%20some%20%3Chttp://purl.obolibrary.org/obo/{id}%3E&amp;direct=false&amp;includeDeprecated=false&amp;includeEquivalent=true".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)


## Owlery Neurons Synaptic
Description: Neurons with synaptic terminals here

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "http://owl.virtualflybrain.org/kbs/vfb/subclasses?" + "object=%3Chttp://purl.obolibrary.org/obo/FBbt_00005106%3E%20and%20%3Chttp://purl.obolibrary.org/obo/RO_0002130%3E%20some%20%3Chttp://purl.obolibrary.org/obo/{id}%3E&amp;direct=false&amp;includeDeprecated=false&amp;includeEquivalent=true".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)


## Owlery Neurons Presynaptic
Description: Neurons with presynaptic terminals here

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "http://owl.virtualflybrain.org/kbs/vfb/subclasses?" + "object=%3Chttp://purl.obolibrary.org/obo/FBbt_00005106%3E%20and%20%3Chttp://purl.obolibrary.org/obo/RO_0002113%3E%20some%20%3Chttp://purl.obolibrary.org/obo/{id}%3E&amp;direct=false&amp;includeDeprecated=false&amp;includeEquivalent=true".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)


## Owlery Neurons Postsynaptic
Description: Neurons with postsynaptic terminals here

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "http://owl.virtualflybrain.org/kbs/vfb/subclasses?" + "object=%3Chttp://purl.obolibrary.org/obo/FBbt_00005106%3E%20and%20%3Chttp://purl.obolibrary.org/obo/RO_0002110%3E%20some%20%3Chttp://purl.obolibrary.org/obo/{id}%3E&amp;direct=false&amp;includeDeprecated=false&amp;includeEquivalent=true".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)


## Owlery Neuron classes fasciculating here
Description: Neuron classes fasciculating here

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "http://owl.virtualflybrain.org/kbs/vfb/subclasses?" + "object=%3Chttp://purl.obolibrary.org/obo/FBbt_00005106%3E%20and%20%3Chttp://purl.obolibrary.org/obo/RO_0002101%3E%20some%20%3Chttp://purl.obolibrary.org/obo/{id}%3E&amp;direct=false&amp;includeDeprecated=false&amp;includeEquivalent=true".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)


## Owlery tracts in
Description: tracts in

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "http://owl.virtualflybrain.org/kbs/vfb/subclasses?" + "object=%3Chttp://purl.obolibrary.org/obo/FBbt_00005099%3E%20and%20%3Chttp://purl.obolibrary.org/obo/RO_0002134%3E%20some%20%3Chttp://purl.obolibrary.org/obo/{id}%3E&amp;direct=false&amp;includeDeprecated=false&amp;includeEquivalent=true".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)


## Owlery Subclasses of
Description: Subclasses of $NAME

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "http://owl.virtualflybrain.org/kbs/vfb/subclasses?" + "object=%3Chttp://purl.obolibrary.org/obo/{id}%3E&amp;direct=false&amp;includeDeprecated=false&amp;includeEquivalent=true".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)


## Owlery Lineage Clones
Description: Lineage clones found in

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "http://owl.virtualflybrain.org/kbs/vfb/subclasses?" + "object=%3Chttp://purl.obolibrary.org/obo/FBbt_00007683%3E%20and%20%3Chttp://purl.obolibrary.org/obo/RO_0002131%3E%20some%20%3Chttp://purl.obolibrary.org/obo/{id}%3E&amp;direct=false&amp;includeDeprecated=false&amp;includeEquivalent=true".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)


## subClassOf cell that overlaps some X
Description: subClassOf cell that overlaps some X

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "http://owl.virtualflybrain.org/kbs/vfb/subclasses?" + "object=%3Chttp://purl.obolibrary.org/obo/FBbt_00007002%3E%20and%20(%20%3Chttp://purl.obolibrary.org/obo/RO_0002131%3E%20some%20%3Chttp://purl.obolibrary.org/obo/{id}%3E%20)&amp;direct=false&amp;includeDeprecated=false&amp;includeEquivalent=true".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)


## subClassOf overlaps some X
Description: subClassOf overlaps some X

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "http://owl.virtualflybrain.org/kbs/vfb/subclasses?" + "object=%3Chttp://purl.obolibrary.org/obo/RO_0002131%3E%20some%20%3Chttp://purl.obolibrary.org/obo/{id}%3E&amp;direct=false&amp;includeDeprecated=false&amp;includeEquivalent=false".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)


## subClassOf cell overlaps some X
Description: subClassOf cell overlaps some X

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "http://owl.virtualflybrain.org/kbs/vfb/subclasses?" + "object=%3Chttp://purl.obolibrary.org/obo/FBbt_00007002%3E%20%20that%20%3Chttp://purl.obolibrary.org/obo/RO_0002131%3E%20some%20%3Chttp://purl.obolibrary.org/obo/{id}%3E&amp;direct=false&amp;includeDeprecated=false&amp;includeEquivalent=false".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)


## Owlery Images of neurons with some part here (clustered)
Description: Images of neurons with some part here (clustered)

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "http://owl.virtualflybrain.org/kbs/vfb/instances?" + "object=%3Chttp://purl.obolibrary.org/obo/C888C3DB-AEFA-447F-BD4C-858DFE33DBE7%3E%20some%20(%3Chttp://purl.obolibrary.org/obo/FBbt_00005106%3E%20and%20%3Chttp://purl.obolibrary.org/obo/RO_0002131%3E%20some%20%3Chttp://purl.obolibrary.org/obo/{id}%3E)&amp;direct=false&amp;includeDeprecated=false".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)


## Owlery Images of neurons with some part here
Description: Images of neurons with some part here

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "http://owl.virtualflybrain.org/kbs/vfb/instances?" + "object=%3Chttp://purl.obolibrary.org/obo/FBbt_00005106%3E%20and%20%3Chttp://purl.obolibrary.org/obo/RO_0002131%3E%20some%20%3Chttp://purl.obolibrary.org/obo/{id}%3E&amp;direct=false&amp;includeDeprecated=false".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)


## Owlery individual parts
Description: Find individuals that are part of some X (useful for finding expression pattern parts).

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "http://owl.virtualflybrain.org/kbs/vfb/instances?" + "object=%3Chttp://purl.obolibrary.org/obo/BFO_0000050%3E%20some%20%3Chttp://virtualflybrain.org/reports/{id}%3E&amp;direct=false&amp;includeDeprecated=false".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)


## Get anat_query
Description: Fetches essential details.

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "https://solr.virtualflybrain.org/solr/vfb_json/select?" + "&quot;params&quot;:{{&quot;defType&quot;:&quot;edismax&quot;,&quot;fl&quot;:&quot;anat_query,anat_image_query&quot;,&quot;indent&quot;:&quot;true&quot;,&quot;q.op&quot;:&quot;OR&quot;,&quot;q&quot;:&quot;id:*&quot;,&quot;fq&quot;:&quot;{{!terms f=id}}{ids}&quot;,&quot;rows&quot;:&quot;999999&quot;}}".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)


## Get user NBLAST results
Description: Fetches user NBLAST results

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "https://solr.virtualflybrain.org/solr/vfb_json/select?" + "&quot;params&quot;:{{&quot;defType&quot;:&quot;edismax&quot;,&quot;fl&quot;:&quot;upload_nblast_query&quot;,&quot;indent&quot;:&quot;true&quot;,&quot;q.op&quot;:&quot;OR&quot;,&quot;q&quot;:&quot;id:{id}&quot;,&quot;qf&quot;:&quot;id&quot;,&quot;rows&quot;:&quot;99&quot;}}".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)


## Get term info
Description: Fetches essential details.

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "https://solr.virtualflybrain.org/solr/vfb_json/select?" + "&quot;params&quot;:{{&quot;defType&quot;:&quot;edismax&quot;,&quot;fl&quot;:&quot;term_info&quot;,&quot;indent&quot;:&quot;true&quot;,&quot;q.op&quot;:&quot;OR&quot;,&quot;q&quot;:&quot;id:{id}&quot;,&quot;qf&quot;:&quot;id&quot;,&quot;rows&quot;:&quot;1&quot;}}".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)


## Get cached VFB_JSON for Term
Description: Get cached VFB_JSON for Term

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "https://solr.virtualflybrain.org/solr/vfb_json/select?" + "&quot;params&quot;:{{&quot;defType&quot;:&quot;edismax&quot;,&quot;fl&quot;:&quot;term_info&quot;,&quot;indent&quot;:&quot;true&quot;,&quot;q.op&quot;:&quot;OR&quot;,&quot;q&quot;:&quot;{id}&quot;,&quot;qf&quot;:&quot;id&quot;,&quot;rows&quot;:&quot;1&quot;}}".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)


## Get anat_2_ep_query
Description: Fetches essential details.

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "https://solr.virtualflybrain.org/solr/vfb_json/select?" + "&quot;params&quot;:{{&quot;defType&quot;:&quot;edismax&quot;,&quot;fl&quot;:&quot;anat_2_ep_query&quot;,&quot;indent&quot;:&quot;true&quot;,&quot;q.op&quot;:&quot;OR&quot;,&quot;q&quot;:&quot;id:*&quot;,&quot;fq&quot;:&quot;{{!terms f=id}}{ids}&quot;,&quot;rows&quot;:&quot;999999&quot;}}".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)


## Get template_2_datasets_query
Description: Fetches essential details.

In [ ]:

# Query execution for Solr or Owlery
import requests
import time

id = 'YOUR_SINGLE_ID_HERE'  # Replace with an actual ID
ids = ['ID1', 'ID2']  # Replace with actual IDs
query_url = "https://solr.virtualflybrain.org/solr/vfb_json/select?" + "&quot;params&quot;:{{&quot;defType&quot;:&quot;edismax&quot;,&quot;fl&quot;:&quot;template_2_datasets_query&quot;,&quot;indent&quot;:&quot;true&quot;,&quot;q.op&quot;:&quot;OR&quot;,&quot;q&quot;:&quot;id:*&quot;,&quot;fq&quot;:&quot;{{!terms f=id}}{ids}&quot;,&quot;rows&quot;:&quot;999999&quot;}}".format(id=id, ids=','.join(ids))

start_time = time.time()
response = requests.get(query_url)
end_time = time.time()
print('Status Code:', response.status_code)
print('Response:', response.text)
print('Time taken:', end_time - start_time)
